In [0]:
from google.colab import drive
drive.mount('/content/drive')

# 데이터 로드 & 함수 선언

In [0]:
from tensorflow import keras
import tensorflow as tf

data = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = data.load_data()

test_labels = test_labels.reshape(-1,1)
train_labels = train_labels.reshape(-1,1)

train_images = train_images / 255
test_images = test_images /255

def make_conv_twice(input_layer, num_filter, keep_prob, layer_num,  batch_prob, filter_size=3, strides=1):
#   w1 = tf.Variable(tf.random_normal([filter_size,filter_size,1,num_filter]) )
  w1 = tf.get_variable("w1_"+str(layer_num), shape=[filter_size,filter_size,1,num_filter], initializer=tf.contrib.layers.xavier_initializer())
  L1 = tf.nn.conv2d(input_layer, w1, strides=[1,strides,strides,1], padding='SAME')
  L1 = tf.layers.batch_normalization(L1, center=True, scale=True, training=batch_prob)
  L1 = tf.nn.relu(L1)
#   w1_1 = tf.Variable(tf.random_normal([filter_size,filter_size,num_filter,num_filter]))
  w1_1 = tf.get_variable("w2_"+str(layer_num), shape=[filter_size,filter_size,num_filter,num_filter], initializer=tf.contrib.layers.xavier_initializer())
  L1_1 = tf.nn.conv2d(L1, w1_1, strides=[1,strides,strides,1], padding='SAME') 
  L1_1 = tf.nn.relu(L1_1)
  L1_1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
  res = L1_1
#   res = tf.nn.dropout(L1_1, keep_prob=keep_prob)
  
  return res

def make_conv_once(input_layer, num_filter, keep_prob, layer_num, batch_prob, filter_size=3, strides=1):
  w1 = tf.get_variable("w1_"+str(layer_num), shape=[filter_size,filter_size,1,num_filter], initializer=tf.contrib.layers.xavier_initializer())
  L1 = tf.nn.conv2d(input_layer, w1, strides=[1,strides,strides,1], padding='SAME')
  L1 = tf.nn.relu(L1)
  L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
  res = L1
#   res = tf.nn.dropout(L1, keep_prob=keep_prob)
  
  return res

def make_conv_once_not_same(input_layer, num_filter, keep_prob, layer_num, batch_prob, filter_size=3, strides=1):
  w1 = tf.get_variable("w1_"+str(layer_num), shape=[filter_size,filter_size,1,num_filter], initializer=tf.contrib.layers.xavier_initializer())
  w1 = tf.Variable(tf.random_normal([filter_size,filter_size,1,num_filter]))
  L1 = tf.nn.conv2d(input_layer, w1, strides=[1,strides,strides,1], padding="VALID")
  L1 = tf.nn.relu(L1)
  L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding="VALID")
  res = L1
#   res = tf.nn.dropout(L1, keep_prob=keep_prob)
  
  return res

def make_conv_once_not_same_wo_pooling(input_layer, num_filter, keep_prob, layer_num, batch_prob, filter_size=3, strides=1):
  w1 = tf.get_variable("w1_"+str(layer_num), shape=[filter_size,filter_size,1,num_filter], initializer=tf.contrib.layers.xavier_initializer())
  L1 = tf.nn.conv2d(input_layer, w1, strides=[1,strides,strides,1], padding="VALID")
  L1 = tf.nn.relu(L1)
  res = L1
#   res = tf.nn.dropout(L1, keep_prob=keep_prob)
  
  return res

def make_conv_once_wo_pooling(input_layer, num_filter, keep_prob, layer_num, batch_prob, padding="SAME", filter_size=3, strides=1):
  w1 = tf.get_variable("w1_"+str(layer_num), shape=[filter_size,filter_size,1,num_filter], initializer=tf.contrib.layers.xavier_initializer())
  L1 = tf.nn.conv2d(input_layer, w1, strides=[1,strides,strides,1], padding=padding)
  L1 = tf.nn.relu(L1)
  res = L1
#   res = tf.nn.dropout(L1, keep_prob=keep_prob)
  
  return res

def my_inception_model(input_layer, keep_prob, batch_prob, filter_num_list):
  w1 = tf.Variable(tf.random_normal([1,1,filter_num_list[6],filter_num_list[0]]))
  l1 = tf.nn.conv2d(input_layer, w1, strides=[1,1,1,1], padding="SAME")
  
  w2_1 = tf.Variable(tf.random_normal([1,1,filter_num_list[6],filter_num_list[1]]))
  l2_1 = tf.nn.conv2d(input_layer, w2_1, strides=[1,1,1,1], padding="SAME")
  w2_2 = tf.Variable(tf.random_normal([3, 3, filter_num_list[1], filter_num_list[2]]))
  l2_2 = tf.nn.conv2d(l2_1, w2_2, strides=[1,1,1,1], padding="SAME")
  
  w3_1 = tf.Variable(tf.random_normal([1,1,filter_num_list[6],filter_num_list[3]]))
  l3_1 = tf.nn.conv2d(input_layer, w3_1, strides=[1,1,1,1], padding="SAME")
  w3_2 = tf.Variable(tf.random_normal([5, 5, filter_num_list[3], filter_num_list[4]]))
  l3_2 = tf.nn.conv2d(l3_1, w3_2, strides=[1,1,1,1], padding="SAME")
  
  l4_1 = tf.nn.max_pool(input_layer, ksize=[1,3,3,1], strides=[1,1,1,1], padding="SAME")
  w4_1 = tf.Variable(tf.random_normal([1,1,filter_num_list[6],filter_num_list[5]]))
  l4_2 = tf.nn.conv2d(l4_1, w4_1, strides=[1,1,1,1], padding="SAME")
  
  res = tf.concat([l1, l2_2, l3_2, l4_2], 3) # input의 shape가 [-1,28,28,1] 이라고 가정
  
  return res # 다시 이 return된 layer를 inception 모델에 돌리기!

def my_inception_model_v2(input_layer, keep_prob, batch_prob, filter_num_list, layer_num):
  w1 = tf.get_variable("w1_"+str(layer_num), shape=[1,1,filter_num_list[6],filter_num_list[0]], initializer=tf.contrib.layers.xavier_initializer())
  l1 = tf.nn.conv2d(input_layer, w1, strides=[1,1,1,1], padding="SAME")
  l1 = tf.layers.batch_normalization(l1, center=True, scale=True, training=batch_prob)
  
  w2_1 = tf.get_variable("w2_1_"+str(layer_num), shape=[1,1,filter_num_list[6],filter_num_list[1]], initializer=tf.contrib.layers.xavier_initializer())
  l2_1 = tf.nn.conv2d(input_layer, w2_1, strides=[1,1,1,1], padding="SAME")
  l2_1 = tf.layers.batch_normalization(l2_1, center=True, scale=True, training=batch_prob)
  w2_2 = tf.get_variable("w2_2_"+str(layer_num), shape=[3,3,filter_num_list[1],filter_num_list[2]], initializer=tf.contrib.layers.xavier_initializer())
  l2_2 = tf.nn.conv2d(l2_1, w2_2, strides=[1,1,1,1], padding="SAME")
  l2_2 = tf.layers.batch_normalization(l2_2, center=True, scale=True, training=batch_prob)
  
  w3_1 = tf.get_variable("w3_1_"+str(layer_num), shape=[1,1,filter_num_list[6],filter_num_list[3]], initializer=tf.contrib.layers.xavier_initializer())
  l3_1 = tf.nn.conv2d(input_layer, w3_1, strides=[1,1,1,1], padding="SAME")
  l3_1 = tf.layers.batch_normalization(l3_1, center=True, scale=True, training=batch_prob)
  w3_2 = tf.get_variable("w3_2_"+str(layer_num), shape=[5,5,filter_num_list[3],filter_num_list[4]], initializer=tf.contrib.layers.xavier_initializer())
  l3_2 = tf.nn.conv2d(l3_1, w3_2, strides=[1,1,1,1], padding="SAME")
  l3_2 = tf.layers.batch_normalization(l3_2, center=True, scale=True, training=batch_prob)
  
  l4_1 = tf.nn.max_pool(input_layer, ksize=[1,3,3,1], strides=[1,1,1,1], padding="SAME")
  w4_1 = tf.get_variable("w4_1_"+str(layer_num), shape=[1,1,filter_num_list[6],filter_num_list[5]], initializer=tf.contrib.layers.xavier_initializer())
  l4_2 = tf.nn.conv2d(l4_1, w4_1, strides=[1,1,1,1], padding="SAME")
  l4_2 = tf.layers.batch_normalization(l4_2, center=True, scale=True, training=batch_prob)
  
  res = tf.concat([l1, l2_2, l3_2, l4_2], 3) # input의 shape가 [-1,28,28,1] 이라고 가정
  
  return res # 다시 이 return된 layer를 inception 모델에 돌리기!

# 이 함수에는 pooling이 포함되어 있지 않습니다.
def my_resNet_model(input_layer, keep_prob, batch_prob, filter_num_list, layer_num):
  bn1 = tf.layers.batch_normalization(input_layer, center=True, scale=True, training=batch_prob)
  bn1 = tf.nn.relu(bn1)
  w1 = tf.get_variable("w1_"+str(layer_num), shape=[3,3,filter_num_list[0],filter_num_list[1]], initializer=tf.contrib.layers.xavier_initializer())
  h1 = tf.nn.conv2d(input_layer, w1, strides=[1,1,1,1], padding="SAME")
  h1 = tf.nn.dropout(h1, keep_prob=keep_prob)
  bn2 = tf.layers.batch_normalization(h1, center=True, scale=True, training=batch_prob)
  bn2 = tf.nn.relu(bn2)
  w2 = tf.get_variable("w2_"+str(layer_num), shape=[3,3,filter_num_list[1],filter_num_list[2]], initializer=tf.contrib.layers.xavier_initializer())
  h2 = tf.nn.conv2d(bn2, w2, strides=[1,1,1,1], padding="SAME")
  
  if filter_num_list[0]!=filter_num_list[-1]:
    us = tf.get_variable("us_"+str(layer_num), shape=[1,1,filter_num_list[0],filter_num_list[2]], initializer=tf.contrib.layers.xavier_initializer())
    input_layer = tf.nn.conv2d(input_layer, us, strides=[1,1,1,1], padding="SAME")
  
  return input_layer + h2
  
  
def shuffleData(images, labels):
  tmp = np.concatenate([images.reshape(-1,28*28),labels],axis=1)
  tmpd = pd.DataFrame(tmp).sample(frac=1)
  ret = tmpd.values.reshape(-1,785)
  retImg = ret[:,:-1].reshape(-1,28,28)
  retlab = ret[:,-1].reshape(-1,1)
  return retImg, retlab

In [0]:
train_images.shape

(60000, 28, 28)

# 텐서 보드

In [0]:
## 텐서 보드 생성.
from tensorboardcolab import * 
from datetime import datetime

current_time = str(datetime.now().timestamp())
train_log_dir = 'logs/tensorboard/train/' + current_time

tbc = TensorBoardColab(graph_path = train_log_dir) # To create a tensorboardcolab object it will automatically creat a link
writer = tbc.get_writer() # To create a FileWriter
writer.add_graph(tf.get_default_graph()) # add the graph 
writer.flush()


## 텐서 보드 생성.
from tensorboardcolab import * 

Using TensorFlow backend.


Wait for 8 seconds...


W0815 22:37:03.505860 140165723359104 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tensorboardcolab/core.py:49: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



TensorBoard link:
https://7f0b5604.ngrok.io


# 실제 실행 부분

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf

tf.reset_default_graph()

learning_rate = 1e-3
training_epochs = 32
batch_size = 100

root = '/content/drive/My Drive/Colab Notebooks/MLpy'
load_file = 'model_mnist_fashion_cnn_inception_0813_BN_100epoch.ckpt'
load_file = root + load_file
save_file = root + 'model_mnist_fashion_cnn_inception_0813_BN_100epoch_mod.ckpt'

x = tf.placeholder(tf.float32, [None,28,28])
x_img = tf.reshape(x, [-1, 28, 28, 1])
y = tf.placeholder(tf.int32, [None,1])
y_ohe = tf.reshape(tf.one_hot(y,10),[-1,10])

batch_prob = tf.placeholder(tf.bool)

keep_prob = tf.placeholder(tf.float32)
keep_prob_flatten = tf.placeholder(tf.float32)

h1 = make_conv_once_wo_pooling(x_img, 64, keep_prob, 1, batch_prob)
h1_1 = make_conv_once_wo_pooling(h1, 192, keep_prob, 1_1, batch_prob)

iList1 = [64,96,128,16,32,32,192]
h2 = my_inception_model_v2(h1_1, keep_prob, batch_prob, iList1, 2)

iList2 = [128,128,192,32,96,64,256]
h3 = my_inception_model_v2(h2, keep_prob,batch_prob,iList2,3)
h3 = tf.nn.max_pool(h3, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

iList3 = [192,96,208,16,48,64,480]
h4 = my_inception_model_v2(h3, keep_prob,batch_prob, iList3, 4)

iList4 = [160,112,224,24,64,64,512]
h5 = my_inception_model_v2(h4, keep_prob, batch_prob, iList4, 5)

iList5 = [128,128,256,24,64,64,512]
h6 = my_inception_model_v2(h5, keep_prob, batch_prob, iList5, 6)

iList6 = [112,144,288,32,64,64,512]
h7 = my_inception_model_v2(h6, keep_prob, batch_prob, iList6, 7)

iList7 = [256,160,320,32,128,128,528]
h8 = my_inception_model_v2(h7, keep_prob, batch_prob, iList7, 8)
h8 = tf.nn.max_pool(h8, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

iList8 = [256,160,320,32,128,128,832]
h9 = my_inception_model_v2(h8, keep_prob, batch_prob, iList8, 9)

iList9 = [384,192,384,48,128,128,832]
h10 = my_inception_model_v2(h9, keep_prob, batch_prob, iList9, 10)

h11 = tf.nn.avg_pool(h10, ksize=[1,7,7,1], strides=[1,7,7,1], padding="SAME")
h11_flat = tf.reshape(h11, [-1,1024])

fcw1 = tf.get_variable("fcw1", shape=[1024,10], initializer=tf.contrib.layers.xavier_initializer())
fcb1 = tf.zeros([10])

logits = tf.matmul(h11_flat,fcw1) + fcb1
logits = tf.nn.dropout(logits, keep_prob=keep_prob_flatten)

prediction = tf.argmax(logits, 1)

c_pre = tf.equal(tf.argmax(logits,1), tf.argmax(y_ohe,1))
acc = tf.reduce_mean(tf.cast(c_pre, tf.float32))

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_ohe, logits=logits))
train = tf.train.AdamOptimizer(learning_rate).minimize(cost)

W0814 16:34:39.582724 140365125961600 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0814 16:34:39.634604 140365125961600 deprecation.py:323] From <ipython-input-1-79d4041f1000>:94: batch_normalization (from tensorflow.python.layers.normalization) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
W0814 16:34:43.635774 140365125961600 deprecation.py:506] From <ipython-input-3-d31cd5ee0471>:65: calling dropout (from tensorflow.pyt

In [0]:
print("training start")

import numpy as np

saver = tf.train.Saver()
my_data = np.zeros([1,28*28])

with tf.Session() as sess:
  saver.restore(sess, load_file)
#   sess.run(tf.global_variables_initializer())
  for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(train_images.shape[0]/batch_size)
    sf_images, sf_labels = shuffleData(train_images, train_labels)
    for i in range(total_batch):
      batch_xs, batch_ys = sf_images[batch_size*i:batch_size*(i+1),:,:], sf_labels[batch_size*i:batch_size*(i+1)]
      myfeed = {x:batch_xs, y:batch_ys, keep_prob:0.7, keep_prob_flatten:0.5, batch_prob:True}
      cv, accv,  _ = sess.run([cost, acc, train], feed_dict=myfeed)
      avg_cost += cv/total_batch
      if i == 900 or i == 1800:
        print("batch: {}, train_acc: {}, avg_cost: {}".format(i,accv,avg_cost))
    print("*"*50)  
    print("epoch: {}, train_acc: {}, avg_cost: {}".format(epoch,accv,avg_cost))
    print("*"*50)
    saver.save(sess, save_file)
  
  final_acu = 0
  for num in range(100):
    acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:False})
    final_acu += acu
  final_acu = final_acu / 100
  print("final_acc: {}".format(final_acu))
#   predict = sess.run(prediction, feed_dict={x:my_data, keep_prob: 1, keep_prob_flatten: 1})
#   print("my_data shape: {}".format(my_data.shape))
#   print(predict[0])
#   print(type(acu))

training start
**************************************************
epoch: 0, train_acc: 0.5199999809265137, avg_cost: 0.9997651163736981
**************************************************
**************************************************
epoch: 1, train_acc: 0.5199999809265137, avg_cost: 0.9849679580330843
**************************************************
**************************************************
epoch: 2, train_acc: 0.47999998927116394, avg_cost: 0.9779691456754994
**************************************************
**************************************************
epoch: 3, train_acc: 0.5299999713897705, avg_cost: 0.9669578102231022
**************************************************
**************************************************
epoch: 4, train_acc: 0.6299999952316284, avg_cost: 0.9599478222926454
**************************************************
**************************************************
epoch: 5, train_acc: 0.5600000023841858, avg_cost: 0.9584405269225437
***

# validation : batch_prob : True!

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf

tf.reset_default_graph()

learning_rate = 1e-3
training_epochs = 32
batch_size = 100

root = '/content/drive/My Drive/Colab Notebooks/MLpy'
load_file = 'model_mnist_fashion_cnn_inception_0813_BN_100epoch_mod.ckpt'
load_file = root + load_file
save_file = root + 'model_mnist_fashion_cnn_inception_0813_BN_100epoch_mod.ckpt'

x = tf.placeholder(tf.float32, [None,28,28])
x_img = tf.reshape(x, [-1, 28, 28, 1])
y = tf.placeholder(tf.int32, [None,1])
y_ohe = tf.reshape(tf.one_hot(y,10),[-1,10])

batch_prob = tf.placeholder(tf.bool)

keep_prob = tf.placeholder(tf.float32)
keep_prob_flatten = tf.placeholder(tf.float32)

h1 = make_conv_once_wo_pooling(x_img, 64, keep_prob, 1, batch_prob)
h1_1 = make_conv_once_wo_pooling(h1, 192, keep_prob, 1_1, batch_prob)

iList1 = [64,96,128,16,32,32,192]
h2 = my_inception_model_v2(h1_1, keep_prob, batch_prob, iList1, 2)

iList2 = [128,128,192,32,96,64,256]
h3 = my_inception_model_v2(h2, keep_prob,batch_prob,iList2,3)
h3 = tf.nn.max_pool(h3, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

iList3 = [192,96,208,16,48,64,480]
h4 = my_inception_model_v2(h3, keep_prob,batch_prob, iList3, 4)

iList4 = [160,112,224,24,64,64,512]
h5 = my_inception_model_v2(h4, keep_prob, batch_prob, iList4, 5)

iList5 = [128,128,256,24,64,64,512]
h6 = my_inception_model_v2(h5, keep_prob, batch_prob, iList5, 6)

iList6 = [112,144,288,32,64,64,512]
h7 = my_inception_model_v2(h6, keep_prob, batch_prob, iList6, 7)

iList7 = [256,160,320,32,128,128,528]
h8 = my_inception_model_v2(h7, keep_prob, batch_prob, iList7, 8)
h8 = tf.nn.max_pool(h8, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

iList8 = [256,160,320,32,128,128,832]
h9 = my_inception_model_v2(h8, keep_prob, batch_prob, iList8, 9)

iList9 = [384,192,384,48,128,128,832]
h10 = my_inception_model_v2(h9, keep_prob, batch_prob, iList9, 10)

h11 = tf.nn.avg_pool(h10, ksize=[1,7,7,1], strides=[1,7,7,1], padding="SAME")
h11_flat = tf.reshape(h11, [-1,1024])

fcw1 = tf.get_variable("fcw1", shape=[1024,10], initializer=tf.contrib.layers.xavier_initializer())
fcb1 = tf.zeros([10])

logits = tf.matmul(h11_flat,fcw1) + fcb1
logits = tf.nn.dropout(logits, keep_prob=keep_prob_flatten)

prediction = tf.argmax(logits, 1)

c_pre = tf.equal(tf.argmax(logits,1), tf.argmax(y_ohe,1))
acc = tf.reduce_mean(tf.cast(c_pre, tf.float32))

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_ohe, logits=logits))
train = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [0]:
print("validation start")

import numpy as np

saver = tf.train.Saver()
my_data = np.zeros([1,28*28])

with tf.Session() as sess:
  saver.restore(sess, load_file)
    
  final_acu = 0
  for num in range(10):
    acu = sess.run(acc, feed_dict={x:test_images[1000*num:1000*(num+1),:,:],y:test_labels[1000*num:1000*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:True})
    final_acu += acu
  final_acu = final_acu / 10
  print("final_acc: {}".format(final_acu))
#   predict = sess.run(prediction, feed_dict={x:my_data, keep_prob: 1, keep_prob_flatten: 1})
#   print("my_data shape: {}".format(my_data.shape))
#   print(predict[0])
#   print(type(acu))

validation start
final_acc: 0.9282999992370605


# Batch size가 너무 작았나 ? : 256으로 수정 / 모델 지속적 저장

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf

tf.reset_default_graph()

learning_rate = 1e-3
training_epochs = 10
batch_size = 256

root = '/content/drive/My Drive/Colab Notebooks/MLpy'
# load_file = 'model_mnist_fashion_cnn_inception_0813_BN_100epoch.ckpt'
# load_file = root + load_file
save_file = root + 'model_mnist_fashion_cnn_inception_0813_BN_accuracy_val.ckpt'

x = tf.placeholder(tf.float32, [None,28,28])
x_img = tf.reshape(x, [-1, 28, 28, 1])
y = tf.placeholder(tf.int32, [None,1])
y_ohe = tf.reshape(tf.one_hot(y,10),[-1,10])

batch_prob = tf.placeholder(tf.bool)

keep_prob = tf.placeholder(tf.float32)
keep_prob_flatten = tf.placeholder(tf.float32)

h1 = make_conv_once_wo_pooling(x_img, 64, keep_prob, 1, batch_prob)
h1_1 = make_conv_once_wo_pooling(h1, 192, keep_prob, 1_1, batch_prob)

iList1 = [64,96,128,16,32,32,192]
h2 = my_inception_model_v2(h1_1, keep_prob, batch_prob, iList1, 2)

iList2 = [128,128,192,32,96,64,256]
h3 = my_inception_model_v2(h2, keep_prob,batch_prob,iList2,3)
h3 = tf.nn.max_pool(h3, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

iList3 = [192,96,208,16,48,64,480]
h4 = my_inception_model_v2(h3, keep_prob,batch_prob, iList3, 4)

iList4 = [160,112,224,24,64,64,512]
h5 = my_inception_model_v2(h4, keep_prob, batch_prob, iList4, 5)

iList5 = [128,128,256,24,64,64,512]
h6 = my_inception_model_v2(h5, keep_prob, batch_prob, iList5, 6)

iList6 = [112,144,288,32,64,64,512]
h7 = my_inception_model_v2(h6, keep_prob, batch_prob, iList6, 7)

iList7 = [256,160,320,32,128,128,528]
h8 = my_inception_model_v2(h7, keep_prob, batch_prob, iList7, 8)
h8 = tf.nn.max_pool(h8, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

iList8 = [256,160,320,32,128,128,832]
h9 = my_inception_model_v2(h8, keep_prob, batch_prob, iList8, 9)

iList9 = [384,192,384,48,128,128,832]
h10 = my_inception_model_v2(h9, keep_prob, batch_prob, iList9, 10)

h11 = tf.nn.avg_pool(h10, ksize=[1,7,7,1], strides=[1,7,7,1], padding="SAME")
h11_flat = tf.reshape(h11, [-1,1024])

fcw1 = tf.get_variable("fcw1", shape=[1024,10], initializer=tf.contrib.layers.xavier_initializer())
fcb1 = tf.zeros([10])

logits = tf.matmul(h11_flat,fcw1) + fcb1
logits = tf.nn.dropout(logits, keep_prob=keep_prob_flatten)

prediction = tf.argmax(logits, 1)

c_pre = tf.equal(tf.argmax(logits,1), tf.argmax(y_ohe,1))
acc = tf.reduce_mean(tf.cast(c_pre, tf.float32))

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_ohe, logits=logits))
train = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [0]:
print("training start")

accuracy_list = [0]

import numpy as np

saver = tf.train.Saver()
my_data = np.zeros([1,28*28])

with tf.Session() as sess:
#   saver.restore(sess, load_file)
  sess.run(tf.global_variables_initializer())
  for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(train_images.shape[0]/batch_size)
    sf_images, sf_labels = shuffleData(train_images, train_labels)
    for i in range(total_batch):
      batch_xs, batch_ys = sf_images[batch_size*i:batch_size*(i+1),:,:], sf_labels[batch_size*i:batch_size*(i+1)]
      myfeed = {x:batch_xs, y:batch_ys, keep_prob:0.7, keep_prob_flatten:0.5, batch_prob:True}
      cv, accv,  _ = sess.run([cost, acc, train], feed_dict=myfeed)
      avg_cost += cv/total_batch
      if total_batch%50 == 0:
        print("batch: {}, train_acc: {}, avg_cost: {}".format(i,accv,avg_cost))
    print("*"*50)  
    print("epoch: {}, train_acc: {}, avg_cost: {}".format(epoch,accv,avg_cost))
    print("*"*50)
    final_acu = 0
    for num in range(100):
      acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:False})
      final_acu += acu
    final_acu = final_acu / 100
    print("test_acc: {}".format(final_acu))
    if final_acu > accuracy_list[-1]:
      accuracy_list.append(final_acu)
      saver.save(sess, save_file)
  
#   final_acu = 0
#   for num in range(100):
#     acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:False})
#     final_acu += acu
#   final_acu = final_acu / 100
#   print("final_acc: {}".format(final_acu))
#   predict = sess.run(prediction, feed_dict={x:my_data, keep_prob: 1, keep_prob_flatten: 1})
#   print("my_data shape: {}".format(my_data.shape))
#   print(predict[0])
#   print(type(acu))

training start
**************************************************
epoch: 0, train_acc: 0.44921875, avg_cost: 1.4835703357672079
**************************************************
test_acc: 0.09999999990686774
**************************************************
epoch: 1, train_acc: 0.54296875, avg_cost: 1.1641503050286544
**************************************************
test_acc: 0.09999999990686774
**************************************************
epoch: 2, train_acc: 0.5859375, avg_cost: 1.1033311218787467
**************************************************
test_acc: 0.1669000004976988
**************************************************
epoch: 3, train_acc: 0.5546875, avg_cost: 1.0771177719291456
**************************************************
test_acc: 0.09999999994412065
**************************************************
epoch: 4, train_acc: 0.5703125, avg_cost: 1.0460005257374196
**************************************************
test_acc: 0.09999999994412065
*******************

# Restore해서 계속 학습!

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf

tf.reset_default_graph()

learning_rate = 1e-3
training_epochs = 50
batch_size = 256

root = '/content/drive/My Drive/Colab Notebooks/MLpy'
load_file = 'model_mnist_fashion_cnn_inception_0813_BN_100epoch_mod.ckpt'
load_file = root + load_file
save_file = root + 'model_mnist_fashion_cnn_inception_0813_BN_add_50.ckpt'

x = tf.placeholder(tf.float32, [None,28,28])
x_img = tf.reshape(x, [-1, 28, 28, 1])
y = tf.placeholder(tf.int32, [None,1])
y_ohe = tf.reshape(tf.one_hot(y,10),[-1,10])

batch_prob = tf.placeholder(tf.bool)

keep_prob = tf.placeholder(tf.float32)
keep_prob_flatten = tf.placeholder(tf.float32)

h1 = make_conv_once_wo_pooling(x_img, 64, keep_prob, 1, batch_prob)
h1_1 = make_conv_once_wo_pooling(h1, 192, keep_prob, 1_1, batch_prob)

iList1 = [64,96,128,16,32,32,192]
h2 = my_inception_model_v2(h1_1, keep_prob, batch_prob, iList1, 2)

iList2 = [128,128,192,32,96,64,256]
h3 = my_inception_model_v2(h2, keep_prob,batch_prob,iList2,3)
h3 = tf.nn.max_pool(h3, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

iList3 = [192,96,208,16,48,64,480]
h4 = my_inception_model_v2(h3, keep_prob,batch_prob, iList3, 4)

iList4 = [160,112,224,24,64,64,512]
h5 = my_inception_model_v2(h4, keep_prob, batch_prob, iList4, 5)

iList5 = [128,128,256,24,64,64,512]
h6 = my_inception_model_v2(h5, keep_prob, batch_prob, iList5, 6)

iList6 = [112,144,288,32,64,64,512]
h7 = my_inception_model_v2(h6, keep_prob, batch_prob, iList6, 7)

iList7 = [256,160,320,32,128,128,528]
h8 = my_inception_model_v2(h7, keep_prob, batch_prob, iList7, 8)
h8 = tf.nn.max_pool(h8, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

iList8 = [256,160,320,32,128,128,832]
h9 = my_inception_model_v2(h8, keep_prob, batch_prob, iList8, 9)

iList9 = [384,192,384,48,128,128,832]
h10 = my_inception_model_v2(h9, keep_prob, batch_prob, iList9, 10)

h11 = tf.nn.avg_pool(h10, ksize=[1,7,7,1], strides=[1,7,7,1], padding="SAME")
h11_flat = tf.reshape(h11, [-1,1024])

fcw1 = tf.get_variable("fcw1", shape=[1024,10], initializer=tf.contrib.layers.xavier_initializer())
fcb1 = tf.zeros([10])

logits = tf.matmul(h11_flat,fcw1) + fcb1
logits = tf.nn.dropout(logits, keep_prob=keep_prob_flatten)

prediction = tf.argmax(logits, 1)

c_pre = tf.equal(tf.argmax(logits,1), tf.argmax(y_ohe,1))
acc = tf.reduce_mean(tf.cast(c_pre, tf.float32))

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_ohe, logits=logits))
train = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [0]:
print("training start")

accuracy_list = [0]

import numpy as np

saver = tf.train.Saver()
my_data = np.zeros([1,28*28])

with tf.Session() as sess:
  saver.restore(sess, load_file)
#   sess.run(tf.global_variables_initializer())
  for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(train_images.shape[0]/batch_size)
    sf_images, sf_labels = shuffleData(train_images, train_labels)
    for i in range(total_batch):
      batch_xs, batch_ys = sf_images[batch_size*i:batch_size*(i+1),:,:], sf_labels[batch_size*i:batch_size*(i+1)]
      myfeed = {x:batch_xs, y:batch_ys, keep_prob:0.7, keep_prob_flatten:0.5, batch_prob:True}
      cv, accv,  _ = sess.run([cost, acc, train], feed_dict=myfeed)
      avg_cost += cv/total_batch
      if total_batch%50 == 0:
        print("batch: {}, train_acc: {}, avg_cost: {}".format(i,accv,avg_cost))
    print("*"*50)  
    print("epoch: {}, train_acc: {}, avg_cost: {}".format(epoch,accv,avg_cost))
    print("*"*50)
    final_acu = 0
    for num in range(100):
      acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:True})
      final_acu += acu
    final_acu = final_acu / 100
    print("test_acc: {}".format(final_acu))
    if final_acu > accuracy_list[-1]:
      accuracy_list.append(final_acu)
      saver.save(sess, save_file)
  
#   final_acu = 0
#   for num in range(100):
#     acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:False})
#     final_acu += acu
#   final_acu = final_acu / 100
#   print("final_acc: {}".format(final_acu))
#   predict = sess.run(prediction, feed_dict={x:my_data, keep_prob: 1, keep_prob_flatten: 1})
#   print("my_data shape: {}".format(my_data.shape))
#   print(predict[0])
#   print(type(acu))

training start
**************************************************
epoch: 0, train_acc: 0.5625, avg_cost: 0.8366367936643783
**************************************************
test_acc: 0.9346000015735626
**************************************************
epoch: 1, train_acc: 0.61328125, avg_cost: 0.8306661834064716
**************************************************
test_acc: 0.9338000023365021
**************************************************
epoch: 2, train_acc: 0.58984375, avg_cost: 0.8349245249206184
**************************************************
test_acc: 0.9358000016212463
**************************************************
epoch: 3, train_acc: 0.5625, avg_cost: 0.8325600341344492
**************************************************
test_acc: 0.9371000003814697
**************************************************
epoch: 4, train_acc: 0.60546875, avg_cost: 0.8285959785820071
**************************************************
test_acc: 0.935100000500679
*****************************

KeyboardInterrupt: ignored

# validation : Inception

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf

tf.reset_default_graph()

learning_rate = 1e-3
training_epochs = 50
batch_size = 256

root = '/content/drive/My Drive/Colab Notebooks/MLpy'
load_file = 'model_mnist_fashion_cnn_inception_0813_BN_100epoch_mod.ckpt'
load_file = root + load_file
save_file = root + 'model_mnist_fashion_cnn_inception_0813_BN_add_50.ckpt'

x = tf.placeholder(tf.float32, [None,28,28])
x_img = tf.reshape(x, [-1, 28, 28, 1])
y = tf.placeholder(tf.int32, [None,1])
y_ohe = tf.reshape(tf.one_hot(y,10),[-1,10])

batch_prob = tf.placeholder(tf.bool)

keep_prob = tf.placeholder(tf.float32)
keep_prob_flatten = tf.placeholder(tf.float32)

h1 = make_conv_once_wo_pooling(x_img, 64, keep_prob, 1, batch_prob)
h1_1 = make_conv_once_wo_pooling(h1, 192, keep_prob, 1_1, batch_prob)

iList1 = [64,96,128,16,32,32,192]
h2 = my_inception_model_v2(h1_1, keep_prob, batch_prob, iList1, 2)

iList2 = [128,128,192,32,96,64,256]
h3 = my_inception_model_v2(h2, keep_prob,batch_prob,iList2,3)
h3 = tf.nn.max_pool(h3, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

iList3 = [192,96,208,16,48,64,480]
h4 = my_inception_model_v2(h3, keep_prob,batch_prob, iList3, 4)

iList4 = [160,112,224,24,64,64,512]
h5 = my_inception_model_v2(h4, keep_prob, batch_prob, iList4, 5)

iList5 = [128,128,256,24,64,64,512]
h6 = my_inception_model_v2(h5, keep_prob, batch_prob, iList5, 6)

iList6 = [112,144,288,32,64,64,512]
h7 = my_inception_model_v2(h6, keep_prob, batch_prob, iList6, 7)

iList7 = [256,160,320,32,128,128,528]
h8 = my_inception_model_v2(h7, keep_prob, batch_prob, iList7, 8)
h8 = tf.nn.max_pool(h8, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

iList8 = [256,160,320,32,128,128,832]
h9 = my_inception_model_v2(h8, keep_prob, batch_prob, iList8, 9)

iList9 = [384,192,384,48,128,128,832]
h10 = my_inception_model_v2(h9, keep_prob, batch_prob, iList9, 10)

h11 = tf.nn.avg_pool(h10, ksize=[1,7,7,1], strides=[1,7,7,1], padding="SAME")
h11_flat = tf.reshape(h11, [-1,1024])

fcw1 = tf.get_variable("fcw1", shape=[1024,10], initializer=tf.contrib.layers.xavier_initializer())
fcb1 = tf.zeros([10])

logits = tf.matmul(h11_flat,fcw1) + fcb1
logits = tf.nn.dropout(logits, keep_prob=keep_prob_flatten)

prediction = tf.argmax(logits, 1)

c_pre = tf.equal(tf.argmax(logits,1), tf.argmax(y_ohe,1))
acc = tf.reduce_mean(tf.cast(c_pre, tf.float32))

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_ohe, logits=logits))
train = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [0]:
print("validation start")

accuracy_list = [0]

import numpy as np

saver = tf.train.Saver()
my_data = np.zeros([1,28*28])

with tf.Session() as sess:
  saver.restore(sess, save_file)
#   sess.run(tf.global_variables_initializer())
  final_acu = 0
  for num in range(100):
    acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:True})
    final_acu += acu
  final_acu = final_acu / 100
  print("test_acc: {}".format(final_acu))

  
#   final_acu = 0
#   for num in range(100):
#     acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:False})
#     final_acu += acu
#   final_acu = final_acu / 100
#   print("final_acc: {}".format(final_acu))
#   predict = sess.run(prediction, feed_dict={x:my_data, keep_prob: 1, keep_prob_flatten: 1})
#   print("my_data shape: {}".format(my_data.shape))
#   print(predict[0])
#   print(type(acu))

training start
test_acc: 0.9378000003099441


# ResNet 구현

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf

tf.reset_default_graph()

learning_rate = 0.001
training_epochs = 50
batch_size = 100

root = '/content/drive/My Drive/Colab Notebooks/MLpy'
load_file = 'model_mnist_fashion_cnn_ResNet_0813_BN_100epoch_mod.ckpt'
load_file = root + load_file
save_file = root + 'model_mnist_fashion_cnn_ResNet_0813_BN_100epoch_mod.ckpt'

x = tf.placeholder(tf.float32, [None,28,28])
x_img = tf.reshape(x, [-1, 28, 28, 1])
y = tf.placeholder(tf.int32, [None,1])
y_ohe = tf.reshape(tf.one_hot(y,10),[-1,10])

batch_prob = tf.placeholder(tf.bool)

keep_prob = tf.placeholder(tf.float32)
keep_prob_flatten = tf.placeholder(tf.float32)

h1 = make_conv_once_wo_pooling(x_img, 64, keep_prob, '1a', batch_prob)
h1 = tf.layers.batch_normalization(h1, center=True, scale=True, training=batch_prob)
h1_1 = make_conv_once_wo_pooling(h1, 128, keep_prob, '1b', batch_prob)
h1_1 = tf.layers.batch_normalization(h1_1, center=True, scale=True, training=batch_prob)

iList1 = [128,128,128]
h2 = my_resNet_model(h1_1, keep_prob, batch_prob, iList1, 2)
iList2 = [128,128,128]
h3 = my_resNet_model(h2, keep_prob, batch_prob, iList2, 3)
iList3 = [128,128,128]
h4 = my_resNet_model(h3, keep_prob, batch_prob, iList3, 4)
iList4 = [128,128,256]
h5 = my_resNet_model(h4, keep_prob, batch_prob, iList4, 5)
h5 = tf.nn.max_pool(h5, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

iList5 = [256,256,256]
h6 = my_resNet_model(h5, keep_prob, batch_prob, iList5, 6)
iList6 = [256,256,256]
h7 = my_resNet_model(h6, keep_prob, batch_prob, iList6, 7)
iList7 = [256,256,256]
h8 = my_resNet_model(h7, keep_prob, batch_prob, iList7, 8)
iList8 = [256,256,256]
h9 = my_resNet_model(h8, keep_prob, batch_prob, iList8, 9)
iList9 = [256,256,256]
h10 = my_resNet_model(h9, keep_prob, batch_prob, iList9, 10)
iList10 = [256,256,512]
h11 = my_resNet_model(h10, keep_prob, batch_prob, iList10, 11)
h11 = tf.nn.max_pool(h11, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

iList11 = [512,512,512]
h12 = my_resNet_model(h11, keep_prob, batch_prob, iList11, 12)
iList12 = [512,512,512]
h13 = my_resNet_model(h12, keep_prob, batch_prob, iList12, 13)
iList13 = [512,512,1024]
h14 = my_resNet_model(h13, keep_prob, batch_prob, iList13, 14)

h15 = tf.nn.avg_pool(h14, ksize=[1,7,7,1], strides=[1,7,7,1], padding="SAME")
h15_flat = tf.reshape(h15, [-1,1024])

fcw1 = tf.get_variable("fcw1", shape=[1024,10], initializer=tf.contrib.layers.xavier_initializer())
fcb1 = tf.zeros([10])

logits = tf.matmul(h15_flat,fcw1) + fcb1
# logits = tf.nn.dropout(logits, keep_prob=keep_prob_flatten)

prediction = tf.argmax(logits, 1)

c_pre = tf.equal(tf.argmax(logits,1), tf.argmax(y_ohe,1))
acc = tf.reduce_mean(tf.cast(c_pre, tf.float32))

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_ohe, logits=logits))
train = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [0]:
print("training start")

accuracy_list = [0]

import numpy as np

saver = tf.train.Saver()
my_data = np.zeros([1,28*28])

with tf.Session() as sess:
#   saver.restore(sess, load_file)
  sess.run(tf.global_variables_initializer())
  for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(train_images.shape[0]/batch_size)
    sf_images, sf_labels = shuffleData(train_images, train_labels)
    for i in range(total_batch):
      batch_xs, batch_ys = sf_images[batch_size*i:batch_size*(i+1),:,:], sf_labels[batch_size*i:batch_size*(i+1)]
      myfeed = {x:batch_xs, y:batch_ys, keep_prob:0.7, keep_prob_flatten:0.5, batch_prob:True}
      cv, accv,  _ = sess.run([cost, acc, train], feed_dict=myfeed)
      avg_cost += cv/total_batch
      if i%100 == 0:
        print("batch: {}, train_acc: {}, avg_cost: {}".format(i,accv,avg_cost))
    print("*"*50)  
    print("epoch: {}, train_acc: {}, avg_cost: {}".format(epoch,accv,avg_cost))
    print("*"*50)
    final_acu = 0
    for num in range(100):
      acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:True})
      final_acu += acu
    final_acu = final_acu / 100
    print("test_acc: {}".format(final_acu))
    if final_acu > accuracy_list[-1]:
      accuracy_list.append(final_acu)
      saver.save(sess, save_file)
  
#   final_acu = 0
#   for num in range(100):
#     acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:False})
#     final_acu += acu
#   final_acu = final_acu / 100
#   print("final_acc: {}".format(final_acu))
#   predict = sess.run(prediction, feed_dict={x:my_data, keep_prob: 1, keep_prob_flatten: 1})
#   print("my_data shape: {}".format(my_data.shape))
#   print(predict[0])
#   print(type(acu))

training start
batch: 0, train_acc: 0.07999999821186066, avg_cost: 0.005154190063476563
batch: 100, train_acc: 0.5899999737739563, avg_cost: 1.9964513392249748
batch: 200, train_acc: 0.7099999785423279, avg_cost: 2.2375136198600143
batch: 300, train_acc: 0.7599999904632568, avg_cost: 2.412682707111044
batch: 400, train_acc: 0.8399999737739563, avg_cost: 2.5811256759862133
batch: 500, train_acc: 0.7799999713897705, avg_cost: 2.7341592581073457
**************************************************
epoch: 0, train_acc: 0.8199999928474426, avg_cost: 2.8510153746604936
**************************************************
test_acc: 0.7984000009298324
batch: 0, train_acc: 0.800000011920929, avg_cost: 0.0009434366226196289
batch: 100, train_acc: 0.7300000190734863, avg_cost: 0.19467283651232725
batch: 200, train_acc: 0.9200000166893005, avg_cost: 0.33258394047617906
batch: 300, train_acc: 0.8700000047683716, avg_cost: 0.44289749021331465
batch: 400, train_acc: 0.8500000238418579, avg_cost: 0.560366

# Restore : ResNet

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf

tf.reset_default_graph()

learning_rate = 0.001
training_epochs = 50
batch_size = 100

root = '/content/drive/My Drive/Colab Notebooks/MLpy'
load_file = 'model_mnist_fashion_cnn_ResNet_0813_BN_100epoch_mod.ckpt'
load_file = root + load_file
save_file = root + 'model_mnist_fashion_cnn_ResNet_0813_BN_50epoch_mod_v2.ckpt'

x = tf.placeholder(tf.float32, [None,28,28])
x_img = tf.reshape(x, [-1, 28, 28, 1])
y = tf.placeholder(tf.int32, [None,1])
y_ohe = tf.reshape(tf.one_hot(y,10),[-1,10])

batch_prob = tf.placeholder(tf.bool)

keep_prob = tf.placeholder(tf.float32)
keep_prob_flatten = tf.placeholder(tf.float32)

h1 = make_conv_once_wo_pooling(x_img, 64, keep_prob, '1a', batch_prob)
h1 = tf.layers.batch_normalization(h1, center=True, scale=True, training=batch_prob)
h1_1 = make_conv_once_wo_pooling(h1, 128, keep_prob, '1b', batch_prob)
h1_1 = tf.layers.batch_normalization(h1_1, center=True, scale=True, training=batch_prob)

iList1 = [128,128,128]
h2 = my_resNet_model(h1_1, keep_prob, batch_prob, iList1, 2)
iList2 = [128,128,128]
h3 = my_resNet_model(h2, keep_prob, batch_prob, iList2, 3)
iList3 = [128,128,128]
h4 = my_resNet_model(h3, keep_prob, batch_prob, iList3, 4)
iList4 = [128,128,256]
h5 = my_resNet_model(h4, keep_prob, batch_prob, iList4, 5)
h5 = tf.nn.max_pool(h5, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

iList5 = [256,256,256]
h6 = my_resNet_model(h5, keep_prob, batch_prob, iList5, 6)
iList6 = [256,256,256]
h7 = my_resNet_model(h6, keep_prob, batch_prob, iList6, 7)
iList7 = [256,256,256]
h8 = my_resNet_model(h7, keep_prob, batch_prob, iList7, 8)
iList8 = [256,256,256]
h9 = my_resNet_model(h8, keep_prob, batch_prob, iList8, 9)
iList9 = [256,256,256]
h10 = my_resNet_model(h9, keep_prob, batch_prob, iList9, 10)
iList10 = [256,256,512]
h11 = my_resNet_model(h10, keep_prob, batch_prob, iList10, 11)
h11 = tf.nn.max_pool(h11, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

iList11 = [512,512,512]
h12 = my_resNet_model(h11, keep_prob, batch_prob, iList11, 12)
iList12 = [512,512,512]
h13 = my_resNet_model(h12, keep_prob, batch_prob, iList12, 13)
iList13 = [512,512,1024]
h14 = my_resNet_model(h13, keep_prob, batch_prob, iList13, 14)

h15 = tf.nn.avg_pool(h14, ksize=[1,7,7,1], strides=[1,7,7,1], padding="SAME")
h15_flat = tf.reshape(h15, [-1,1024])

fcw1 = tf.get_variable("fcw1", shape=[1024,10], initializer=tf.contrib.layers.xavier_initializer())
fcb1 = tf.zeros([10])

logits = tf.matmul(h15_flat,fcw1) + fcb1
# logits = tf.nn.dropout(logits, keep_prob=keep_prob_flatten)

prediction = tf.argmax(logits, 1)

c_pre = tf.equal(tf.argmax(logits,1), tf.argmax(y_ohe,1))
acc = tf.reduce_mean(tf.cast(c_pre, tf.float32))

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_ohe, logits=logits))
train = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [0]:
print("training start")

accuracy_list = [0]

import numpy as np

saver = tf.train.Saver()
my_data = np.zeros([1,28*28])

with tf.Session() as sess:
  saver.restore(sess, load_file)
#   sess.run(tf.global_variables_initializer())
  for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(train_images.shape[0]/batch_size)
    sf_images, sf_labels = shuffleData(train_images, train_labels)
    for i in range(total_batch):
      batch_xs, batch_ys = sf_images[batch_size*i:batch_size*(i+1),:,:], sf_labels[batch_size*i:batch_size*(i+1)]
      myfeed = {x:batch_xs, y:batch_ys, keep_prob:0.7, keep_prob_flatten:0.5, batch_prob:True}
      cv, accv,  _ = sess.run([cost, acc, train], feed_dict=myfeed)
      avg_cost += cv/total_batch
      if i%100 == 0:
        print("batch: {}, train_acc: {}, avg_cost: {}".format(i,accv,avg_cost))
    print("*"*50)  
    print("epoch: {}, train_acc: {}, avg_cost: {}".format(epoch,accv,avg_cost))
    print("*"*50)
    final_acu = 0
    for num in range(100):
      acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:True})
      final_acu += acu
    final_acu = final_acu / 100
    print("test_acc: {}".format(final_acu))
    if final_acu > accuracy_list[-1]:
      accuracy_list.append(final_acu)
      saver.save(sess, save_file)
  
#   final_acu = 0
#   for num in range(100):
#     acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:False})
#     final_acu += acu
#   final_acu = final_acu / 100
#   print("final_acc: {}".format(final_acu))
#   predict = sess.run(prediction, feed_dict={x:my_data, keep_prob: 1, keep_prob_flatten: 1})
#   print("my_data shape: {}".format(my_data.shape))
#   print(predict[0])
#   print(type(acu))

training start


W0813 12:14:34.407257 139731626665856 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


batch: 0, train_acc: 0.9800000190734863, avg_cost: 4.3594039355715115e-05
batch: 100, train_acc: 1.0, avg_cost: 0.006564120243225869
batch: 200, train_acc: 0.9800000190734863, avg_cost: 0.012275049300709122
batch: 300, train_acc: 0.9800000190734863, avg_cost: 0.019494537977831596
batch: 400, train_acc: 0.9599999785423279, avg_cost: 0.02573109921310486
batch: 500, train_acc: 0.9900000095367432, avg_cost: 0.03473658621708943
**************************************************
epoch: 0, train_acc: 0.9800000190734863, avg_cost: 0.044063390704589735
**************************************************
test_acc: 0.9285000020265579
batch: 0, train_acc: 1.0, avg_cost: 1.823817069331805e-05
batch: 100, train_acc: 0.9800000190734863, avg_cost: 0.005744013083555426
batch: 200, train_acc: 0.9900000095367432, avg_cost: 0.011082608808840941
batch: 300, train_acc: 0.9900000095367432, avg_cost: 0.01736994489086405
batch: 400, train_acc: 0.9900000095367432, avg_cost: 0.024986772895305577
batch: 500, train

KeyboardInterrupt: ignored

# ResNet : Restore

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf

tf.reset_default_graph()

learning_rate = 0.001
training_epochs = 100
batch_size = 100

root = '/content/drive/My Drive/Colab Notebooks/MLpy'
load_file = 'model_mnist_fashion_cnn_ResNet_0813_BN_50epoch_mod_v2.ckpt'
load_file = root + load_file
save_file = root + 'model_mnist_fashion_cnn_ResNet_0814_BN_150epoch_best_acc.ckpt'
save_file2 = root + 'model_mnist_fashion_cnn_ResNet_0814_BN_150epoch_best_loss.ckpt'

x = tf.placeholder(tf.float32, [None,28,28])
x_img = tf.reshape(x, [-1, 28, 28, 1])
y = tf.placeholder(tf.int32, [None,1])
y_ohe = tf.reshape(tf.one_hot(y,10),[-1,10])

batch_prob = tf.placeholder(tf.bool)

keep_prob = tf.placeholder(tf.float32)
keep_prob_flatten = tf.placeholder(tf.float32)

h1 = make_conv_once_wo_pooling(x_img, 64, keep_prob, '1a', batch_prob)
h1 = tf.layers.batch_normalization(h1, center=True, scale=True, training=batch_prob)
h1_1 = make_conv_once_wo_pooling(h1, 128, keep_prob, '1b', batch_prob)
h1_1 = tf.layers.batch_normalization(h1_1, center=True, scale=True, training=batch_prob)

iList1 = [128,128,128]
h2 = my_resNet_model(h1_1, keep_prob, batch_prob, iList1, 2)
iList2 = [128,128,128]
h3 = my_resNet_model(h2, keep_prob, batch_prob, iList2, 3)
iList3 = [128,128,128]
h4 = my_resNet_model(h3, keep_prob, batch_prob, iList3, 4)
iList4 = [128,128,256]
h5 = my_resNet_model(h4, keep_prob, batch_prob, iList4, 5)
h5 = tf.nn.max_pool(h5, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

iList5 = [256,256,256]
h6 = my_resNet_model(h5, keep_prob, batch_prob, iList5, 6)
iList6 = [256,256,256]
h7 = my_resNet_model(h6, keep_prob, batch_prob, iList6, 7)
iList7 = [256,256,256]
h8 = my_resNet_model(h7, keep_prob, batch_prob, iList7, 8)
iList8 = [256,256,256]
h9 = my_resNet_model(h8, keep_prob, batch_prob, iList8, 9)
iList9 = [256,256,256]
h10 = my_resNet_model(h9, keep_prob, batch_prob, iList9, 10)
iList10 = [256,256,512]
h11 = my_resNet_model(h10, keep_prob, batch_prob, iList10, 11)
h11 = tf.nn.max_pool(h11, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

iList11 = [512,512,512]
h12 = my_resNet_model(h11, keep_prob, batch_prob, iList11, 12)
iList12 = [512,512,512]
h13 = my_resNet_model(h12, keep_prob, batch_prob, iList12, 13)
iList13 = [512,512,1024]
h14 = my_resNet_model(h13, keep_prob, batch_prob, iList13, 14)

h15 = tf.nn.avg_pool(h14, ksize=[1,7,7,1], strides=[1,7,7,1], padding="SAME")
h15_flat = tf.reshape(h15, [-1,1024])

fcw1 = tf.get_variable("fcw1", shape=[1024,10], initializer=tf.contrib.layers.xavier_initializer())
fcb1 = tf.zeros([10])

logits = tf.matmul(h15_flat,fcw1) + fcb1
# logits = tf.nn.dropout(logits, keep_prob=keep_prob_flatten)

prediction = tf.argmax(logits, 1)

c_pre = tf.equal(tf.argmax(logits,1), tf.argmax(y_ohe,1))
acc = tf.reduce_mean(tf.cast(c_pre, tf.float32))

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_ohe, logits=logits))
train = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [0]:
print("training start")

accuracy_list = [0]
loss_list = [100]

import numpy as np

saver = tf.train.Saver()
my_data = np.zeros([1,28*28])

with tf.Session() as sess:
  saver.restore(sess, load_file)
#   sess.run(tf.global_variables_initializer())
  for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(train_images.shape[0]/batch_size)
    sf_images, sf_labels = shuffleData(train_images, train_labels)
    for i in range(total_batch):
      batch_xs, batch_ys = sf_images[batch_size*i:batch_size*(i+1),:,:], sf_labels[batch_size*i:batch_size*(i+1)]
      myfeed = {x:batch_xs, y:batch_ys, keep_prob:0.7, keep_prob_flatten:0.5, batch_prob:True}
      cv, accv,  _ = sess.run([cost, acc, train], feed_dict=myfeed)
      avg_cost += cv/total_batch
      if i%100 == 0:
        print("batch: {}, train_acc: {}, avg_cost: {}".format(i,accv,avg_cost))
    print("*"*50)  
    print("epoch: {}, train_acc: {}, avg_cost: {}".format(epoch,accv,avg_cost))
    print("*"*50)
    final_acu = 0
    for num in range(100):
      acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:True})
      final_acu += acu
    final_acu = final_acu / 100
    print("test_acc: {}".format(final_acu))
    if final_acu > accuracy_list[-1]:
      accuracy_list.append(final_acu)
      saver.save(sess, save_file)
    if avg_cost < loss_list[-1]:
      loss_list.append(avg_cost)
      saver.save(sess, save_file2)
      
print(accuracy_list)
print(loss_list)
  
#   final_acu = 0
#   for num in range(100):
#     acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:False})
#     final_acu += acu
#   final_acu = final_acu / 100
#   print("final_acc: {}".format(final_acu))
#   predict = sess.run(prediction, feed_dict={x:my_data, keep_prob: 1, keep_prob_flatten: 1})
#   print("my_data shape: {}".format(my_data.shape))
#   print(predict[0])
#   print(type(acu))

training start
batch: 0, train_acc: 1.0, avg_cost: 6.078304334854086e-06
batch: 100, train_acc: 1.0, avg_cost: 0.002586844466014214
batch: 200, train_acc: 0.9900000095367432, avg_cost: 0.005905651690203326
batch: 300, train_acc: 1.0, avg_cost: 0.011454919537374627


# ResNet : for 10 classes

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf

tf.reset_default_graph()

learning_rate = 0.001
training_epochs = 10
batch_size = 100

root = '/content/drive/My Drive/Colab Notebooks/MLpy'
load_file = 'model_mnist_fashion_cnn_ResNet_0814_BN.ckpt'
load_file = root + load_file
save_file = root + 'model_mnist_fashion_cnn_ResNet_0814_BN_bst_acc.ckpt'
save_file2 = root + 'model_mnist_fashion_cnn_ResNet_0814_BN_bst_loss.ckpt'

x = tf.placeholder(tf.float32, [None,28,28])
x_img = tf.reshape(x, [-1, 28, 28, 1])
y = tf.placeholder(tf.int32, [None,1])
y_ohe = tf.reshape(tf.one_hot(y,10),[-1,10])

batch_prob = tf.placeholder(tf.bool)

keep_prob = tf.placeholder(tf.float32)
keep_prob_flatten = tf.placeholder(tf.float32)

h1 = make_conv_once_wo_pooling(x_img, 128, keep_prob, '1a', batch_prob)
h1 = tf.layers.batch_normalization(h1, center=True, scale=True, training=batch_prob)

iList1 = [128,128,128]
h2 = my_resNet_model(h1, keep_prob, batch_prob, iList1, 2)
iList2 = [128,128,128]
h3 = my_resNet_model(h2, keep_prob, batch_prob, iList2, 3)
iList3 = [128,128,128]
h4 = my_resNet_model(h3, keep_prob, batch_prob, iList3, 4)
iList4 = [128,128,256]
h5 = my_resNet_model(h4, keep_prob, batch_prob, iList4, 5)
h5 = tf.nn.max_pool(h5, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

iList5 = [256,256,256]
h6 = my_resNet_model(h5, keep_prob, batch_prob, iList5, 6)
iList6 = [256,256,256]
h7 = my_resNet_model(h6, keep_prob, batch_prob, iList6, 7)
iList7 = [256,256,256]
h8 = my_resNet_model(h7, keep_prob, batch_prob, iList7, 8)
iList8 = [256,256,256]
h9 = my_resNet_model(h8, keep_prob, batch_prob, iList8, 9)
iList9 = [256,256,256]
h10 = my_resNet_model(h9, keep_prob, batch_prob, iList9, 10)
iList10 = [256,256,256]
h11 = my_resNet_model(h10, keep_prob, batch_prob, iList10, 11)
h11 = tf.nn.max_pool(h11, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
h12 = tf.nn.avg_pool(h11, ksize=[1,7,7,1], strides=[1,7,7,1], padding="SAME")
h12_flat = tf.reshape(h12, [-1,256])

fcw1 = tf.get_variable("fcw1", shape=[256,10], initializer=tf.contrib.layers.xavier_initializer())
fcb1 = tf.zeros([10])

logits = tf.matmul(h12_flat,fcw1) + fcb1
# logits = tf.nn.dropout(logits, keep_prob=keep_prob_flatten)

prediction = tf.argmax(logits, 1)

c_pre = tf.equal(tf.argmax(logits,1), tf.argmax(y_ohe,1))
acc = tf.reduce_mean(tf.cast(c_pre, tf.float32))

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_ohe, logits=logits))
train = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [0]:
print("training start")

accuracy_list = [0]
loss_list = [100]

import numpy as np

saver = tf.train.Saver()
my_data = np.zeros([1,28*28])

with tf.Session() as sess:
#   saver.restore(sess, load_file)
  sess.run(tf.global_variables_initializer())
  for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(train_images.shape[0]/batch_size)
    sf_images, sf_labels = shuffleData(train_images, train_labels)
    for i in range(total_batch):
      batch_xs, batch_ys = sf_images[batch_size*i:batch_size*(i+1),:,:], sf_labels[batch_size*i:batch_size*(i+1)]
      myfeed = {x:batch_xs, y:batch_ys, keep_prob:0.7, keep_prob_flatten:0.5, batch_prob:True}
      cv, accv,  _ = sess.run([cost, acc, train], feed_dict=myfeed)
      avg_cost += cv/total_batch
      if i%100 == 0:
        print("batch: {}, train_acc: {}, avg_cost: {}".format(i,accv,avg_cost))
    print("*"*50)  
    print("epoch: {}, train_acc: {}, avg_cost: {}".format(epoch,accv,avg_cost))
    print("*"*50)
    final_acu = 0
    for num in range(100):
      acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:True})
      final_acu += acu
    final_acu = final_acu / 100
    print("test_acc: {}".format(final_acu))
    if final_acu > accuracy_list[-1]:
      accuracy_list.append(final_acu)
      saver.save(sess, save_file)
    if avg_cost < loss_list[-1]:
      loss_list.append(avg_cost)
      saver.save(sess, save_file2)
      
print(accuracy_list)
print(loss_list)
  
#   final_acu = 0
#   for num in range(100):
#     acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:False})
#     final_acu += acu
#   final_acu = final_acu / 100
#   print("final_acc: {}".format(final_acu))
#   predict = sess.run(prediction, feed_dict={x:my_data, keep_prob: 1, keep_prob_flatten: 1})
#   print("my_data shape: {}".format(my_data.shape))
#   print(predict[0])
#   print(type(acu))

training start
batch: 0, train_acc: 0.09000000357627869, avg_cost: 0.0061059157053629555
batch: 100, train_acc: 0.6899999976158142, avg_cost: 0.8036426612734793
batch: 200, train_acc: 0.7900000214576721, avg_cost: 0.9371734351913129
batch: 300, train_acc: 0.8199999928474426, avg_cost: 1.0549825007716815
batch: 400, train_acc: 0.8100000023841858, avg_cost: 1.167238837381204
batch: 500, train_acc: 0.7900000214576721, avg_cost: 1.273420827388764
**************************************************
epoch: 0, train_acc: 0.8500000238418579, avg_cost: 1.3607737915714595
**************************************************
test_acc: 0.8178999990224838
batch: 0, train_acc: 0.8399999737739563, avg_cost: 0.0007805954416592916
batch: 100, train_acc: 0.8600000143051147, avg_cost: 0.07833048932254313
batch: 200, train_acc: 0.8500000238418579, avg_cost: 0.15970885174969832
batch: 300, train_acc: 0.8899999856948853, avg_cost: 0.23530263818800445
batch: 400, train_acc: 0.7599999904632568, avg_cost: 0.30812

# Restore : 10 classes

In [0]:
'''
수정사항 
1. 맨 처음 conv layer를 64 / 128 => 128 단층으로 전환
2. #filter를 기존의 128-256-512-1024 => 128-256-256으로 전환
3. 마지막 maxpool 후 바로 average pool 적용
'''

import numpy as np
import pandas as pd
import tensorflow as tf

tf.reset_default_graph()

learning_rate = 0.001
training_epochs = 40
batch_size = 100

root = '/content/drive/My Drive/Colab Notebooks/MLpy'
load_file = 'model_mnist_fashion_cnn_ResNet_0814_BN_bst_acc.ckpt'
load_file = root + load_file
save_file = root + 'model_mnist_fashion_cnn_ResNet_0814_BN_bst_acc.ckpt'
save_file2 = root + 'model_mnist_fashion_cnn_ResNet_0814_BN_bst_acc.ckpt'

x = tf.placeholder(tf.float32, [None,28,28])
x_img = tf.reshape(x, [-1, 28, 28, 1])
y = tf.placeholder(tf.int32, [None,1])
y_ohe = tf.reshape(tf.one_hot(y,10),[-1,10])

batch_prob = tf.placeholder(tf.bool)

keep_prob = tf.placeholder(tf.float32)
keep_prob_flatten = tf.placeholder(tf.float32)

h1 = make_conv_once_wo_pooling(x_img, 128, keep_prob, '1a', batch_prob)
h1 = tf.layers.batch_normalization(h1, center=True, scale=True, training=batch_prob)

iList1 = [128,128,128]
h2 = my_resNet_model(h1, keep_prob, batch_prob, iList1, 2)
iList2 = [128,128,128]
h3 = my_resNet_model(h2, keep_prob, batch_prob, iList2, 3)
iList3 = [128,128,128]
h4 = my_resNet_model(h3, keep_prob, batch_prob, iList3, 4)
iList4 = [128,128,256]
h5 = my_resNet_model(h4, keep_prob, batch_prob, iList4, 5)
h5 = tf.nn.max_pool(h5, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

iList5 = [256,256,256]
h6 = my_resNet_model(h5, keep_prob, batch_prob, iList5, 6)
iList6 = [256,256,256]
h7 = my_resNet_model(h6, keep_prob, batch_prob, iList6, 7)
iList7 = [256,256,256]
h8 = my_resNet_model(h7, keep_prob, batch_prob, iList7, 8)
iList8 = [256,256,256]
h9 = my_resNet_model(h8, keep_prob, batch_prob, iList8, 9)
iList9 = [256,256,256]
h10 = my_resNet_model(h9, keep_prob, batch_prob, iList9, 10)
iList10 = [256,256,256]
h11 = my_resNet_model(h10, keep_prob, batch_prob, iList10, 11)
h11 = tf.nn.max_pool(h11, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
h12 = tf.nn.avg_pool(h11, ksize=[1,7,7,1], strides=[1,7,7,1], padding="SAME")
h12_flat = tf.reshape(h12, [-1,256])

fcw1 = tf.get_variable("fcw1", shape=[256,10], initializer=tf.contrib.layers.xavier_initializer())
fcb1 = tf.zeros([10])

logits = tf.matmul(h12_flat,fcw1) + fcb1
# logits = tf.nn.dropout(logits, keep_prob=keep_prob_flatten)

prediction = tf.argmax(logits, 1)

c_pre = tf.equal(tf.argmax(logits,1), tf.argmax(y_ohe,1))
acc = tf.reduce_mean(tf.cast(c_pre, tf.float32))

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_ohe, logits=logits))
train = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [0]:
print("training start")

accuracy_list = [0]
loss_list = [100]

import numpy as np

saver = tf.train.Saver()
my_data = np.zeros([1,28*28])

with tf.Session() as sess:
  saver.restore(sess, load_file)
#   sess.run(tf.global_variables_initializer())
  for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(train_images.shape[0]/batch_size)
    sf_images, sf_labels = shuffleData(train_images, train_labels)
    for i in range(total_batch):
      batch_xs, batch_ys = sf_images[batch_size*i:batch_size*(i+1),:,:], sf_labels[batch_size*i:batch_size*(i+1)]
      myfeed = {x:batch_xs, y:batch_ys, keep_prob:0.7, keep_prob_flatten:0.5, batch_prob:True}
      cv, accv,  _ = sess.run([cost, acc, train], feed_dict=myfeed)
      avg_cost += cv/total_batch
      if i%100 == 0:
        print("batch: {}, train_acc: {}, avg_cost: {}".format(i,accv,avg_cost))
    print("*"*50)  
    print("epoch: {}, train_acc: {}, avg_cost: {}".format(epoch,accv,avg_cost))
    print("*"*50)
    final_acu = 0
    for num in range(100):
      acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:True})
      final_acu += acu
    final_acu = final_acu / 100
    print("test_acc: {}".format(final_acu))
    if final_acu > accuracy_list[-1]:
      accuracy_list.append(final_acu)
      saver.save(sess, save_file)
    if avg_cost < loss_list[-1]:
      loss_list.append(avg_cost)
      saver.save(sess, save_file2)
      
print(accuracy_list)
print(loss_list)
  
#   final_acu = 0
#   for num in range(100):
#     acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:False})
#     final_acu += acu
#   final_acu = final_acu / 100
#   print("final_acc: {}".format(final_acu))
#   predict = sess.run(prediction, feed_dict={x:my_data, keep_prob: 1, keep_prob_flatten: 1})
#   print("my_data shape: {}".format(my_data.shape))
#   print(predict[0])
#   print(type(acu))

training start


W0814 00:58:49.596909 140036658886528 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


batch: 0, train_acc: 0.9399999976158142, avg_cost: 0.00037185393273830413
batch: 100, train_acc: 0.9200000166893005, avg_cost: 0.03114394760380188
batch: 200, train_acc: 0.9100000262260437, avg_cost: 0.06337655426313481
batch: 300, train_acc: 0.949999988079071, avg_cost: 0.09797682924816999
batch: 400, train_acc: 0.9399999976158142, avg_cost: 0.12937414211531464
batch: 500, train_acc: 0.9100000262260437, avg_cost: 0.1645563122754294
**************************************************
epoch: 0, train_acc: 0.9200000166893005, avg_cost: 0.19895237430309234
**************************************************
test_acc: 0.8990999984741211
batch: 0, train_acc: 0.949999988079071, avg_cost: 0.0003798498461643855
batch: 100, train_acc: 0.9399999976158142, avg_cost: 0.033400914122660955
batch: 200, train_acc: 0.9700000286102295, avg_cost: 0.0648000725731254
batch: 300, train_acc: 0.949999988079071, avg_cost: 0.09450658099725856
batch: 400, train_acc: 0.9599999785423279, avg_cost: 0.1283048984097939

# ResNet : for 10 classes : mod

In [0]:
'''
수정사항 
1. 맨 처음 conv layer를 64 / 128 => 128 단층으로 전환
2. #filter를 기존의 128-256-512-1024 => 128-256-256-256으로 전환
3. 마지막 maxpool 후 바로 average pool 적용
4. 마지막 fc에서 dropout 적용
'''

import numpy as np
import pandas as pd
import tensorflow as tf

tf.reset_default_graph()

learning_rate = 0.001
training_epochs = 50
batch_size = 100

root = '/content/drive/My Drive/Colab Notebooks/MLpy'
load_file = 'model_mnist_fashion_cnn_ResNet_0814_BN_long_bst_cost.ckpt'
load_file = root + load_file
save_file = root + 'model_mnist_fashion_cnn_ResNet_0814_BN_long_bst_acc.ckpt'
save_file2 = root + 'model_mnist_fashion_cnn_ResNet_0814_BN_long_bst_cost.ckpt'

x = tf.placeholder(tf.float32, [None,28,28])
x_img = tf.reshape(x, [-1, 28, 28, 1])
y = tf.placeholder(tf.int32, [None,1])
y_ohe = tf.reshape(tf.one_hot(y,10),[-1,10])

batch_prob = tf.placeholder(tf.bool)

keep_prob = tf.placeholder(tf.float32)
keep_prob_flatten = tf.placeholder(tf.float32)

h1 = make_conv_once_wo_pooling(x_img, 128, keep_prob, '1a', batch_prob)
h1 = tf.layers.batch_normalization(h1, center=True, scale=True, training=batch_prob)

iList1 = [128,128,128]
h2 = my_resNet_model(h1, keep_prob, batch_prob, iList1, 2)
iList2 = [128,128,128]
h3 = my_resNet_model(h2, keep_prob, batch_prob, iList2, 3)
iList3 = [128,128,128]
h4 = my_resNet_model(h3, keep_prob, batch_prob, iList3, 4)
iList4 = [128,128,256]
h5 = my_resNet_model(h4, keep_prob, batch_prob, iList4, 5)
h5 = tf.nn.max_pool(h5, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

iList5 = [256,256,256]
h6 = my_resNet_model(h5, keep_prob, batch_prob, iList5, 6)
iList6 = [256,256,256]
h7 = my_resNet_model(h6, keep_prob, batch_prob, iList6, 7)
iList7 = [256,256,256]
h8 = my_resNet_model(h7, keep_prob, batch_prob, iList7, 8)
iList8 = [256,256,256]
h9 = my_resNet_model(h8, keep_prob, batch_prob, iList8, 9)
iList9 = [256,256,256]
h10 = my_resNet_model(h9, keep_prob, batch_prob, iList9, 10)
iList10 = [256,256,256]
h11 = my_resNet_model(h10, keep_prob, batch_prob, iList10, 11)
h11 = tf.nn.max_pool(h11, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
iList11 = [256,256,256]
h12 = my_resNet_model(h11, keep_prob, batch_prob, iList11, 12)
iList12 = [256,256,256]
h13 = my_resNet_model(h12, keep_prob, batch_prob, iList12, 13)
iList13 = [256,256,256]
h14 = my_resNet_model(h13, keep_prob, batch_prob, iList13, 14)
iList14 = [256,256,256]
h15 = my_resNet_model(h14, keep_prob, batch_prob, iList14, 15)
h15 = tf.nn.avg_pool(h15, ksize=[1,7,7,1], strides=[1,7,7,1], padding="SAME")
h15_flat = tf.reshape(h15, [-1,256])

fcw1 = tf.get_variable("fcw1", shape=[256,10], initializer=tf.contrib.layers.xavier_initializer())
fcb1 = tf.zeros([10])

logits = tf.matmul(h15_flat,fcw1) + fcb1
logits = tf.nn.dropout(logits, keep_prob=keep_prob_flatten)

prediction = tf.argmax(logits, 1)

c_pre = tf.equal(tf.argmax(logits,1), tf.argmax(y_ohe,1))
acc = tf.reduce_mean(tf.cast(c_pre, tf.float32))

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_ohe, logits=logits))
train = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [0]:
print("training start")

accuracy_list = [0]
loss_list = [100]

import numpy as np

saver = tf.train.Saver()
my_data = np.zeros([1,28*28])

with tf.Session() as sess:
#   saver.restore(sess, load_file)
  sess.run(tf.global_variables_initializer())
  for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(train_images.shape[0]/batch_size)
    sf_images, sf_labels = shuffleData(train_images, train_labels)
    for i in range(total_batch):
      batch_xs, batch_ys = sf_images[batch_size*i:batch_size*(i+1),:,:], sf_labels[batch_size*i:batch_size*(i+1)]
      myfeed = {x:batch_xs, y:batch_ys, keep_prob:0.7, keep_prob_flatten:0.5, batch_prob:True}
      cv, accv,  _ = sess.run([cost, acc, train], feed_dict=myfeed)
      avg_cost += cv/total_batch
      if i%100 == 0:
        print("batch: {}, train_acc: {}, avg_cost: {}".format(i,accv,avg_cost))
    print("*"*50)  
    print("epoch: {}, train_acc: {}, avg_cost: {}".format(epoch,accv,avg_cost))
    print("*"*50)
    final_acu = 0
    for num in range(100):
      acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:True})
      final_acu += acu
    final_acu = final_acu / 100
    print("test_acc: {}".format(final_acu))
    if final_acu > accuracy_list[-1]:
      accuracy_list.append(final_acu)
      saver.save(sess, save_file)
    if avg_cost < loss_list[-1]:
      loss_list.append(avg_cost)
      saver.save(sess, save_file2)
      
print(accuracy_list)
print(loss_list)
  
#   final_acu = 0
#   for num in range(100):
#     acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:False})
#     final_acu += acu
#   final_acu = final_acu / 100
#   print("final_acc: {}".format(final_acu))
#   predict = sess.run(prediction, feed_dict={x:my_data, keep_prob: 1, keep_prob_flatten: 1})
#   print("my_data shape: {}".format(my_data.shape))
#   print(predict[0])
#   print(type(acu))

training start
batch: 0, train_acc: 0.10999999940395355, avg_cost: 0.015770867665608725
batch: 100, train_acc: 0.30000001192092896, avg_cost: 1.638140858610471
batch: 200, train_acc: 0.5099999904632568, avg_cost: 1.9410578219095875
batch: 300, train_acc: 0.4099999964237213, avg_cost: 2.2612243402004237
batch: 400, train_acc: 0.44999998807907104, avg_cost: 2.5306990500291184
batch: 500, train_acc: 0.4300000071525574, avg_cost: 2.768920793135959
**************************************************
epoch: 0, train_acc: 0.46000000834465027, avg_cost: 3.017639006177583
**************************************************
test_acc: 0.7518999975919723
batch: 0, train_acc: 0.5199999809265137, avg_cost: 0.0021158464749654136
batch: 100, train_acc: 0.550000011920929, avg_cost: 0.24698745071887968
batch: 200, train_acc: 0.5099999904632568, avg_cost: 0.4783831691741942
batch: 300, train_acc: 0.5199999809265137, avg_cost: 0.69207024137179
batch: 400, train_acc: 0.41999998688697815, avg_cost: 0.94147718

# 18 / 18 /  18

In [0]:
'''
수정사항 
0. 18*2 / 18*2 / 18*2 resnet layer
1. 맨 처음 conv layer를 16 channel 단층으로 전환
2. #filter를 16-32-64로 전환
3. 마지막 maxpool 후 바로 average pool 적용
4. 마지막 fc에서 dropout 적용
'''

import numpy as np
import pandas as pd
import tensorflow as tf

tf.reset_default_graph()

learning_rate = 0.001
training_epochs = 50
batch_size = 100

root = '/content/drive/My Drive/Colab Notebooks/MLpy'
load_file = 'model_mnist_fashion_cnn_ResNet_0814_BN_54l_bst_cost.ckpt'
load_file = root + load_file
save_file = root + 'model_mnist_fashion_cnn_ResNet_0814_BN_54l_bst_acc.ckpt'
save_file2 = root + 'model_mnist_fashion_cnn_ResNet_0814_BN_54l_bst_cost.ckpt'

x = tf.placeholder(tf.float32, [None,28,28])
x_img = tf.reshape(x, [-1, 28, 28, 1])
y = tf.placeholder(tf.int32, [None,1])
y_ohe = tf.reshape(tf.one_hot(y,10),[-1,10])

batch_prob = tf.placeholder(tf.bool)

keep_prob = tf.placeholder(tf.float32)
keep_prob_flatten = tf.placeholder(tf.float32)

h1 = make_conv_once_wo_pooling(x_img, 16, keep_prob, '1a', batch_prob)
h1 = tf.layers.batch_normalization(h1, center=True, scale=True, training=batch_prob)

iList1 = [16,16,16]
iList1_1 = [16,16,32]
iList2 = [32,32,32]
iList2_1 = [32,32,64]
iList3 = [64,64,64]
hidden_layer_list = []
h2 = my_resNet_model(h1, keep_prob, batch_prob, iList1, 2)
hidden_layer_list.append(h2)
for i in range(3,19):
  hl = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList1, i)
  hidden_layer_list.append(hl)

h19 = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList1_1, 19)
h19 = tf.nn.max_pool(h19, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
hidden_layer_list.append(h19)

for i in range(20,37):
  hl = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList2, i)
  hidden_layer_list.append(hl)

h37 = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList2_1, 37)
h37 = tf.nn.max_pool(h37, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
hidden_layer_list.append(h37)

for i in range(38,55):
  hl = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList3, i)
  hidden_layer_list.append(hl)

h55 = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList3, 55)
h55 = tf.nn.max_pool(h55, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
hidden_layer_list.append(h55)
  
h56 = tf.nn.avg_pool(h55, ksize=[1,7,7,1], strides=[1,7,7,1], padding="SAME")
h56_flat = tf.reshape(h56, [-1,64])

fcw1 = tf.get_variable("fcw1", shape=[64,10], initializer=tf.contrib.layers.xavier_initializer())
fcb1 = tf.zeros([10])

logits = tf.matmul(h56_flat,fcw1) + fcb1
logits = tf.nn.dropout(logits, keep_prob=keep_prob_flatten)

prediction = tf.argmax(logits, 1)

c_pre = tf.equal(tf.argmax(logits,1), tf.argmax(y_ohe,1))
acc = tf.reduce_mean(tf.cast(c_pre, tf.float32))

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_ohe, logits=logits))
train = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [0]:
print("training start")

accuracy_list = [0]
loss_list = [100]

import numpy as np

saver = tf.train.Saver()
my_data = np.zeros([1,28*28])

with tf.Session() as sess:
#   saver.restore(sess, load_file)
  sess.run(tf.global_variables_initializer())
  for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(train_images.shape[0]/batch_size)
    sf_images, sf_labels = shuffleData(train_images, train_labels)
    for i in range(total_batch):
      batch_xs, batch_ys = sf_images[batch_size*i:batch_size*(i+1),:,:], sf_labels[batch_size*i:batch_size*(i+1)]
      myfeed = {x:batch_xs, y:batch_ys, keep_prob:0.7, keep_prob_flatten:0.5, batch_prob:True}
      cv, accv,  _ = sess.run([cost, acc, train], feed_dict=myfeed)
      avg_cost += cv/total_batch
      if i%100 == 0:
        print("batch: {}, train_acc: {}, avg_cost: {}".format(i,accv,avg_cost))
    print("*"*50)  
    print("epoch: {}, train_acc: {}, avg_cost: {}".format(epoch,accv,avg_cost))
    print("*"*50)
    final_acu = 0
    for num in range(100):
      acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:True})
      final_acu += acu
    final_acu = final_acu / 100
    print("test_acc: {}".format(final_acu))
    if final_acu > accuracy_list[-1]:
      accuracy_list.append(final_acu)
      saver.save(sess, save_file)
    if avg_cost < loss_list[-1]:
      loss_list.append(avg_cost)
      saver.save(sess, save_file2)
      
print(accuracy_list)
print(loss_list)
  
#   final_acu = 0
#   for num in range(100):
#     acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:False})
#     final_acu += acu
#   final_acu = final_acu / 100
#   print("final_acc: {}".format(final_acu))
#   predict = sess.run(prediction, feed_dict={x:my_data, keep_prob: 1, keep_prob_flatten: 1})
#   print("my_data shape: {}".format(my_data.shape))
#   print(predict[0])
#   print(type(acu))

training start
batch: 0, train_acc: 0.10999999940395355, avg_cost: 0.03721494992574056
batch: 100, train_acc: 0.36000001430511475, avg_cost: 0.6860487838586172
batch: 200, train_acc: 0.47999998927116394, avg_cost: 0.9641724280516307
batch: 300, train_acc: 0.4399999976158142, avg_cost: 1.209512250026068
batch: 400, train_acc: 0.5699999928474426, avg_cost: 1.436432611147565
batch: 500, train_acc: 0.5799999833106995, avg_cost: 1.670493098696075
**************************************************
epoch: 0, train_acc: 0.5299999713897705, avg_cost: 1.8978204399347331
**************************************************
test_acc: 0.8102999967336655
batch: 0, train_acc: 0.6200000047683716, avg_cost: 0.0016428598761558534
batch: 100, train_acc: 0.5099999904632568, avg_cost: 0.2233051987489063
batch: 200, train_acc: 0.5099999904632568, avg_cost: 0.4373609753449757
batch: 300, train_acc: 0.5299999713897705, avg_cost: 0.6507124202450119
batch: 400, train_acc: 0.46000000834465027, avg_cost: 0.85158570

# Cont. : Restore ResNet

In [0]:
'''
수정사항 
0. 18*2 / 18*2 / 18*2 resnet layer
1. 맨 처음 conv layer를 16 channel 단층으로 전환
2. #filter를 16-32-64로 전환
3. 마지막 maxpool 후 바로 average pool 적용
4. 마지막 fc에서 dropout 적용
'''

import numpy as np
import pandas as pd
import tensorflow as tf

tf.reset_default_graph()

learning_rate = 0.001
training_epochs = 25
batch_size = 100

root = '/content/drive/My Drive/Colab Notebooks/MLpy'
load_file = 'model_mnist_fashion_cnn_ResNet_0814_BN_54l_bst_cost.ckpt'
load_file = root + load_file
save_file = root + 'model_mnist_fashion_cnn_ResNet_0814_BN_54l_bst_acc.ckpt'
save_file2 = root + 'model_mnist_fashion_cnn_ResNet_0814_BN_54l_bst_cost.ckpt'

x = tf.placeholder(tf.float32, [None,28,28])
x_img = tf.reshape(x, [-1, 28, 28, 1])
y = tf.placeholder(tf.int32, [None,1])
y_ohe = tf.reshape(tf.one_hot(y,10),[-1,10])

batch_prob = tf.placeholder(tf.bool)

keep_prob = tf.placeholder(tf.float32)
keep_prob_flatten = tf.placeholder(tf.float32)

h1 = make_conv_once_wo_pooling(x_img, 16, keep_prob, '1a', batch_prob)
h1 = tf.layers.batch_normalization(h1, center=True, scale=True, training=batch_prob)

iList1 = [16,16,16]
iList1_1 = [16,16,32]
iList2 = [32,32,32]
iList2_1 = [32,32,64]
iList3 = [64,64,64]
hidden_layer_list = []
h2 = my_resNet_model(h1, keep_prob, batch_prob, iList1, 2)
hidden_layer_list.append(h2)
for i in range(3,19):
  hl = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList1, i)
  hidden_layer_list.append(hl)

h19 = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList1_1, 19)
h19 = tf.nn.max_pool(h19, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
hidden_layer_list.append(h19)

for i in range(20,37):
  hl = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList2, i)
  hidden_layer_list.append(hl)

h37 = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList2_1, 37)
h37 = tf.nn.max_pool(h37, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
hidden_layer_list.append(h37)

for i in range(38,55):
  hl = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList3, i)
  hidden_layer_list.append(hl)

h55 = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList3, 55)
h55 = tf.nn.max_pool(h55, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
hidden_layer_list.append(h55)
  
h56 = tf.nn.avg_pool(h55, ksize=[1,7,7,1], strides=[1,7,7,1], padding="SAME")
h56_flat = tf.reshape(h56, [-1,64])

fcw1 = tf.get_variable("fcw1", shape=[64,10], initializer=tf.contrib.layers.xavier_initializer())
fcb1 = tf.zeros([10])

logits = tf.matmul(h56_flat,fcw1) + fcb1
logits = tf.nn.dropout(logits, keep_prob=keep_prob_flatten)

prediction = tf.argmax(logits, 1)

c_pre = tf.equal(tf.argmax(logits,1), tf.argmax(y_ohe,1))
acc = tf.reduce_mean(tf.cast(c_pre, tf.float32))

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_ohe, logits=logits))
train = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [0]:
print("training start")

accuracy_list = [0]
loss_list = [100]

import numpy as np

saver = tf.train.Saver()
my_data = np.zeros([1,28*28])

with tf.Session() as sess:
  saver.restore(sess, load_file)
#   sess.run(tf.global_variables_initializer())
  for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(train_images.shape[0]/batch_size)
    sf_images, sf_labels = shuffleData(train_images, train_labels)
    for i in range(total_batch):
      batch_xs, batch_ys = sf_images[batch_size*i:batch_size*(i+1),:,:], sf_labels[batch_size*i:batch_size*(i+1)]
      myfeed = {x:batch_xs, y:batch_ys, keep_prob:0.7, keep_prob_flatten:0.5, batch_prob:True}
      cv, accv,  _ = sess.run([cost, acc, train], feed_dict=myfeed)
      avg_cost += cv/total_batch
      if i%100 == 0:
        print("batch: {}, train_acc: {}, avg_cost: {}".format(i,accv,avg_cost))
    print("*"*50)  
    print("epoch: {}, train_acc: {}, avg_cost: {}".format(epoch,accv,avg_cost))
    print("*"*50)
    final_acu = 0
    for num in range(100):
      acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:True})
      final_acu += acu
    final_acu = final_acu / 100
    print("test_acc: {}".format(final_acu))
    if final_acu > accuracy_list[-1]:
      accuracy_list.append(final_acu)
      saver.save(sess, save_file)
    if avg_cost < loss_list[-1]:
      loss_list.append(avg_cost)
      saver.save(sess, save_file2)
      
print(accuracy_list)
print(loss_list)
  
#   final_acu = 0
#   for num in range(100):
#     acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:False})
#     final_acu += acu
#   final_acu = final_acu / 100
#   print("final_acc: {}".format(final_acu))
#   predict = sess.run(prediction, feed_dict={x:my_data, keep_prob: 1, keep_prob_flatten: 1})
#   print("my_data shape: {}".format(my_data.shape))
#   print(predict[0])
#   print(type(acu))

training start
batch: 0, train_acc: 0.5699999928474426, avg_cost: 0.001661586860815684
batch: 100, train_acc: 0.6499999761581421, avg_cost: 0.14738524446884793
batch: 200, train_acc: 0.5199999809265137, avg_cost: 0.29692100803057364
batch: 300, train_acc: 0.6299999952316284, avg_cost: 0.4422937098145486
batch: 400, train_acc: 0.6499999761581421, avg_cost: 0.5899402118722595
batch: 500, train_acc: 0.5699999928474426, avg_cost: 0.7372463192542391
**************************************************
epoch: 0, train_acc: 0.5099999904632568, avg_cost: 0.883684594730536
**************************************************
test_acc: 0.9254000020027161
batch: 0, train_acc: 0.6299999952316284, avg_cost: 0.0014896883567174276
batch: 100, train_acc: 0.5699999928474426, avg_cost: 0.1473659917712212
batch: 200, train_acc: 0.5699999928474426, avg_cost: 0.2952321117122968
batch: 300, train_acc: 0.5199999809265137, avg_cost: 0.44170152455568307
batch: 400, train_acc: 0.46000000834465027, avg_cost: 0.58606

#Cont. : fc에 dropout 없앰

In [0]:
'''
수정사항 
0. 18*2 / 18*2 / 18*2 resnet layer
1. 맨 처음 conv layer를 16 channel 단층으로 전환
2. #filter를 16-32-64로 전환
3. 마지막 maxpool 후 바로 average pool 적용
4. 마지막 fc에서 dropout 적용
'''

import numpy as np
import pandas as pd
import tensorflow as tf

tf.reset_default_graph()

learning_rate = 0.00001
training_epochs = 50
batch_size = 100

root = '/content/drive/My Drive/Colab Notebooks/MLpy'
load_file = 'model_mnist_fashion_cnn_ResNet_0814_BN_54l_bst_cost_flatten.ckpt'
load_file = root + load_file
save_file = root + 'model_mnist_fashion_cnn_ResNet_0814_BN_54l_bst_acc_flatten.ckpt'
save_file2 = root + 'model_mnist_fashion_cnn_ResNet_0814_BN_54l_bst_cost_flatten.ckpt'

x = tf.placeholder(tf.float32, [None,28,28])
x_img = tf.reshape(x, [-1, 28, 28, 1])
y = tf.placeholder(tf.int32, [None,1])
y_ohe = tf.reshape(tf.one_hot(y,10),[-1,10])

batch_prob = tf.placeholder(tf.bool)

keep_prob = tf.placeholder(tf.float32)
keep_prob_flatten = tf.placeholder(tf.float32)

h1 = make_conv_once_wo_pooling(x_img, 16, keep_prob, '1a', batch_prob)
h1 = tf.layers.batch_normalization(h1, center=True, scale=True, training=batch_prob)

iList1 = [16,16,16]
iList1_1 = [16,16,32]
iList2 = [32,32,32]
iList2_1 = [32,32,64]
iList3 = [64,64,64]
hidden_layer_list = []
h2 = my_resNet_model(h1, keep_prob, batch_prob, iList1, 2)
hidden_layer_list.append(h2)
for i in range(3,19):
  hl = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList1, i)
  hidden_layer_list.append(hl)

h19 = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList1_1, 19)
h19 = tf.nn.max_pool(h19, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
hidden_layer_list.append(h19)

for i in range(20,37):
  hl = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList2, i)
  hidden_layer_list.append(hl)

h37 = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList2_1, 37)
h37 = tf.nn.max_pool(h37, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
hidden_layer_list.append(h37)

for i in range(38,55):
  hl = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList3, i)
  hidden_layer_list.append(hl)

h55 = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList3, 55)
h55 = tf.nn.max_pool(h55, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
hidden_layer_list.append(h55)
  
h56 = tf.nn.avg_pool(h55, ksize=[1,7,7,1], strides=[1,7,7,1], padding="SAME")
h56_flat = tf.reshape(h56, [-1,64])

fcw1 = tf.get_variable("fcw1", shape=[64,10], initializer=tf.contrib.layers.xavier_initializer())
fcb1 = tf.zeros([10])

logits = tf.matmul(h56_flat,fcw1) + fcb1
logits = tf.nn.dropout(logits, keep_prob=keep_prob_flatten)

prediction = tf.argmax(logits, 1)

c_pre = tf.equal(tf.argmax(logits,1), tf.argmax(y_ohe,1))
acc = tf.reduce_mean(tf.cast(c_pre, tf.float32))

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_ohe, logits=logits))
train = tf.train.AdamOptimizer(learning_rate).minimize(cost)

W0815 02:26:09.855189 140235330328448 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0815 02:26:09.888265 140235330328448 deprecation.py:323] From <ipython-input-3-4f14eef2ef83>:37: batch_normalization (from tensorflow.python.layers.normalization) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
W0815 02:26:17.466892 140235330328448 deprecation.py:506] From <ipython-input-3-4f14eef2ef83>:78: calling dropout (from tensorflow.pyt

In [0]:
print("training start")

accuracy_list = [0]
loss_list = [100]

import numpy as np

saver = tf.train.Saver()
my_data = np.zeros([1,28*28])

with tf.Session() as sess:
  saver.restore(sess, load_file)
#   sess.run(tf.global_variables_initializer())
  for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(train_images.shape[0]/batch_size)
    sf_images, sf_labels = shuffleData(train_images, train_labels)
    for i in range(total_batch):
      batch_xs, batch_ys = sf_images[batch_size*i:batch_size*(i+1),:,:], sf_labels[batch_size*i:batch_size*(i+1)]
      myfeed = {x:batch_xs, y:batch_ys, keep_prob:0.7, keep_prob_flatten:1, batch_prob:True}
      cv, accv,  _ = sess.run([cost, acc, train], feed_dict=myfeed)
      avg_cost += cv/total_batch
      if i%100 == 0:
        print("batch: {}, train_acc: {}, avg_cost: {}".format(i,accv,avg_cost))
    print("*"*50)  
    print("epoch: {}, train_acc: {}, avg_cost: {}".format(epoch,accv,avg_cost))
    print("*"*50)
    final_acu = 0
    for num in range(100):
      acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:True})
      final_acu += acu
    final_acu = final_acu / 100
    print("test_acc: {}".format(final_acu))
    if final_acu > accuracy_list[-1]:
      accuracy_list.append(final_acu)
      saver.save(sess, save_file)
    if avg_cost < loss_list[-1]:
      loss_list.append(avg_cost)
      saver.save(sess, save_file2)
      
print(accuracy_list)
print(loss_list)
  
#   final_acu = 0
#   for num in range(100):
#     acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:False})
#     final_acu += acu
#   final_acu = final_acu / 100
#   print("final_acc: {}".format(final_acu))
#   predict = sess.run(prediction, feed_dict={x:my_data, keep_prob: 1, keep_prob_flatten: 1})
#   print("my_data shape: {}".format(my_data.shape))
#   print(predict[0])
#   print(type(acu))

training start


W0815 02:26:26.133930 140235330328448 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


batch: 0, train_acc: 1.0, avg_cost: 1.9868214481041e-12
batch: 100, train_acc: 1.0, avg_cost: 6.9953072889816725e-09
batch: 200, train_acc: 1.0, avg_cost: 1.626304685734917e-08
batch: 300, train_acc: 1.0, avg_cost: 2.0188967496300353e-08
batch: 400, train_acc: 1.0, avg_cost: 2.9212616543169028e-08
batch: 500, train_acc: 1.0, avg_cost: 3.175574337044291e-08
**************************************************
epoch: 0, train_acc: 1.0, avg_cost: 3.568564394799497e-08
**************************************************
test_acc: 0.9307000029087067
batch: 0, train_acc: 1.0, avg_cost: 3.9736428962082e-12
batch: 100, train_acc: 1.0, avg_cost: 6.772985368108664e-09
batch: 200, train_acc: 1.0, avg_cost: 1.2814785205576377e-08
batch: 300, train_acc: 1.0, avg_cost: 1.8751240078070594e-08
batch: 400, train_acc: 1.0, avg_cost: 2.195199231983897e-08
batch: 500, train_acc: 1.0, avg_cost: 2.8188441197526024e-08
**************************************************
epoch: 1, train_acc: 1.0, avg_cost: 3.0864

# dropout을 적용한 ResNet

In [0]:
'''
수정사항 
0. 18*2 / 18*2 / 18*2 resnet layer
1. 맨 처음 conv layer를 16 channel 단층으로 전환
2. #filter를 16-32-64로 전환
3. 마지막 maxpool 후 바로 average pool 적용
4. 마지막 fc에서 dropout 적용
'''

import numpy as np
import pandas as pd
import tensorflow as tf

tf.reset_default_graph()

learning_rate = 0.00001
training_epochs = 50
batch_size = 100

root = '/content/drive/My Drive/Colab Notebooks/MLpy'
load_file = 'model_mnist_fashion_cnn_ResNet_0814_BN_54l_bst_cost.ckpt'
load_file = root + load_file
save_file = root + 'model_mnist_fashion_cnn_ResNet_0814_BN_54l_bst_acc.ckpt'
save_file2 = root + 'model_mnist_fashion_cnn_ResNet_0814_BN_54l_bst_cost.ckpt'

x = tf.placeholder(tf.float32, [None,28,28])
x_img = tf.reshape(x, [-1, 28, 28, 1])
y = tf.placeholder(tf.int32, [None,1])
y_ohe = tf.reshape(tf.one_hot(y,10),[-1,10])

batch_prob = tf.placeholder(tf.bool)

keep_prob = tf.placeholder(tf.float32)
keep_prob_flatten = tf.placeholder(tf.float32)

h1 = make_conv_once_wo_pooling(x_img, 16, keep_prob, '1a', batch_prob)
h1 = tf.layers.batch_normalization(h1, center=True, scale=True, training=batch_prob)

iList1 = [16,16,16]
iList1_1 = [16,16,32]
iList2 = [32,32,32]
iList2_1 = [32,32,64]
iList3 = [64,64,64]
hidden_layer_list = []
h2 = my_resNet_model(h1, keep_prob, batch_prob, iList1, 2)
hidden_layer_list.append(h2)
for i in range(3,19):
  hl = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList1, i)
  hidden_layer_list.append(hl)

h19 = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList1_1, 19)
h19 = tf.nn.max_pool(h19, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
hidden_layer_list.append(h19)

for i in range(20,37):
  hl = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList2, i)
  hidden_layer_list.append(hl)

h37 = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList2_1, 37)
h37 = tf.nn.max_pool(h37, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
hidden_layer_list.append(h37)

for i in range(38,55):
  hl = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList3, i)
  hidden_layer_list.append(hl)

h55 = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList3, 55)
h55 = tf.nn.max_pool(h55, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
hidden_layer_list.append(h55)
  
h56 = tf.nn.avg_pool(h55, ksize=[1,7,7,1], strides=[1,7,7,1], padding="SAME")
h56_flat = tf.reshape(h56, [-1,64])

fcw1 = tf.get_variable("fcw1", shape=[64,10], initializer=tf.contrib.layers.xavier_initializer())
fcb1 = tf.zeros([10])

logits = tf.matmul(h56_flat,fcw1) + fcb1
logits = tf.nn.dropout(logits, keep_prob=keep_prob_flatten)

prediction = tf.argmax(logits, 1)

c_pre = tf.equal(tf.argmax(logits,1), tf.argmax(y_ohe,1))
acc = tf.reduce_mean(tf.cast(c_pre, tf.float32))

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_ohe, logits=logits))
train = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [0]:
print("training start")

accuracy_list = [0]
loss_list = [100]

import numpy as np

saver = tf.train.Saver()
my_data = np.zeros([1,28*28])

with tf.Session() as sess:
  saver.restore(sess, load_file)
#   sess.run(tf.global_variables_initializer())
  for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(train_images.shape[0]/batch_size)
    sf_images, sf_labels = shuffleData(train_images, train_labels)
    for i in range(total_batch):
      batch_xs, batch_ys = sf_images[batch_size*i:batch_size*(i+1),:,:], sf_labels[batch_size*i:batch_size*(i+1)]
      myfeed = {x:batch_xs, y:batch_ys, keep_prob:0.7, keep_prob_flatten:0.5, batch_prob:True}
      cv, accv,  _ = sess.run([cost, acc, train], feed_dict=myfeed)
      avg_cost += cv/total_batch
      if i%100 == 0:
        print("batch: {}, train_acc: {}, avg_cost: {}".format(i,accv,avg_cost))
    print("*"*50)  
    print("epoch: {}, train_acc: {}, avg_cost: {}".format(epoch,accv,avg_cost))
    print("*"*50)
    final_acu = 0
    for num in range(100):
      acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:True})
      final_acu += acu
    final_acu = final_acu / 100
    print("test_acc: {}".format(final_acu))
    if final_acu > accuracy_list[-1]:
      accuracy_list.append(final_acu)
      saver.save(sess, save_file)
    if avg_cost < loss_list[-1]:
      loss_list.append(avg_cost)
      saver.save(sess, save_file2)
      
print(accuracy_list)
print(loss_list)
  
#   final_acu = 0
#   for num in range(100):
#     acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:False})
#     final_acu += acu
#   final_acu = final_acu / 100
#   print("final_acc: {}".format(final_acu))
#   predict = sess.run(prediction, feed_dict={x:my_data, keep_prob: 1, keep_prob_flatten: 1})
#   print("my_data shape: {}".format(my_data.shape))
#   print(predict[0])
#   print(type(acu))

training start
batch: 0, train_acc: 0.6100000143051147, avg_cost: 0.0013672564427057903
batch: 100, train_acc: 0.7099999785423279, avg_cost: 0.14226533919572823
batch: 200, train_acc: 0.5600000023841858, avg_cost: 0.28245035350322734
batch: 300, train_acc: 0.6000000238418579, avg_cost: 0.42375343014796574
batch: 400, train_acc: 0.5400000214576721, avg_cost: 0.5656221961975098
batch: 500, train_acc: 0.6399999856948853, avg_cost: 0.7051856792966523
**************************************************
epoch: 0, train_acc: 0.6299999952316284, avg_cost: 0.8448312785228091
**************************************************
test_acc: 0.9285000032186508
batch: 0, train_acc: 0.5400000214576721, avg_cost: 0.0016069595019022624
batch: 100, train_acc: 0.6399999856948853, avg_cost: 0.14254403491814932
batch: 200, train_acc: 0.6000000238418579, avg_cost: 0.28085944294929527
batch: 300, train_acc: 0.5600000023841858, avg_cost: 0.4218203474084537
batch: 400, train_acc: 0.5400000214576721, avg_cost: 0.56

KeyboardInterrupt: ignored

# dropout 0.9

In [0]:
'''
수정사항 
0. 18*2 / 18*2 / 18*2 resnet layer
1. 맨 처음 conv layer를 16 channel 단층으로 전환
2. #filter를 16-32-64로 전환
3. 마지막 maxpool 후 바로 average pool 적용
4. 마지막 fc에서 dropout 적용
'''

import numpy as np
import pandas as pd
import tensorflow as tf

tf.reset_default_graph()

learning_rate = 0.00001
training_epochs = 50
batch_size = 100

root = '/content/drive/My Drive/Colab Notebooks/MLpy'
load_file = 'model_mnist_fashion_cnn_ResNet_0814_BN_54l_bst_cost.ckpt'
load_file = root + load_file
save_file = root + 'model_mnist_fashion_cnn_ResNet_0814_BN_54l_bst_acc.ckpt'
save_file2 = root + 'model_mnist_fashion_cnn_ResNet_0814_BN_54l_bst_cost_dr0.9.ckpt'

x = tf.placeholder(tf.float32, [None,28,28])
x_img = tf.reshape(x, [-1, 28, 28, 1])
y = tf.placeholder(tf.int32, [None,1])
y_ohe = tf.reshape(tf.one_hot(y,10),[-1,10])

batch_prob = tf.placeholder(tf.bool)

keep_prob = tf.placeholder(tf.float32)
keep_prob_flatten = tf.placeholder(tf.float32)

h1 = make_conv_once_wo_pooling(x_img, 16, keep_prob, '1a', batch_prob)
h1 = tf.layers.batch_normalization(h1, center=True, scale=True, training=batch_prob)

iList1 = [16,16,16]
iList1_1 = [16,16,32]
iList2 = [32,32,32]
iList2_1 = [32,32,64]
iList3 = [64,64,64]
hidden_layer_list = []
h2 = my_resNet_model(h1, keep_prob, batch_prob, iList1, 2)
hidden_layer_list.append(h2)
for i in range(3,19):
  hl = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList1, i)
  hidden_layer_list.append(hl)

h19 = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList1_1, 19)
h19 = tf.nn.max_pool(h19, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
hidden_layer_list.append(h19)

for i in range(20,37):
  hl = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList2, i)
  hidden_layer_list.append(hl)

h37 = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList2_1, 37)
h37 = tf.nn.max_pool(h37, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
hidden_layer_list.append(h37)

for i in range(38,55):
  hl = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList3, i)
  hidden_layer_list.append(hl)

h55 = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList3, 55)
h55 = tf.nn.max_pool(h55, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
hidden_layer_list.append(h55)
  
h56 = tf.nn.avg_pool(h55, ksize=[1,7,7,1], strides=[1,7,7,1], padding="SAME")
h56_flat = tf.reshape(h56, [-1,64])

fcw1 = tf.get_variable("fcw1", shape=[64,10], initializer=tf.contrib.layers.xavier_initializer())
fcb1 = tf.zeros([10])

logits = tf.matmul(h56_flat,fcw1) + fcb1
logits = tf.nn.dropout(logits, keep_prob=keep_prob_flatten)

prediction = tf.argmax(logits, 1)

c_pre = tf.equal(tf.argmax(logits,1), tf.argmax(y_ohe,1))
acc = tf.reduce_mean(tf.cast(c_pre, tf.float32))

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_ohe, logits=logits))
train = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [0]:
print("training start")

accuracy_list = [0]
loss_list = [100]

import numpy as np

saver = tf.train.Saver()
my_data = np.zeros([1,28*28])

with tf.Session() as sess:
  saver.restore(sess, load_file)
#   sess.run(tf.global_variables_initializer())
  for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(train_images.shape[0]/batch_size)
    sf_images, sf_labels = shuffleData(train_images, train_labels)
    for i in range(total_batch):
      batch_xs, batch_ys = sf_images[batch_size*i:batch_size*(i+1),:,:], sf_labels[batch_size*i:batch_size*(i+1)]
      myfeed = {x:batch_xs, y:batch_ys, keep_prob:1, keep_prob_flatten:0.9, batch_prob:True}
      cv, accv,  _ = sess.run([cost, acc, train], feed_dict=myfeed)
      avg_cost += cv/total_batch
      if i%100 == 0:
        print("batch: {}, train_acc: {}, avg_cost: {}".format(i,accv,avg_cost))
    print("*"*50)  
    print("epoch: {}, train_acc: {}, avg_cost: {}".format(epoch,accv,avg_cost))
    print("*"*50)
    final_acu = 0
    for num in range(100):
      acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:True})
      final_acu += acu
    final_acu = final_acu / 100
    print("test_acc: {}".format(final_acu))
    if final_acu > accuracy_list[-1]:
      accuracy_list.append(final_acu)
      saver.save(sess, save_file)
    if avg_cost < loss_list[-1]:
      loss_list.append(avg_cost)
      saver.save(sess, save_file2)
      
print(accuracy_list)
print(loss_list)
  
#   final_acu = 0
#   for num in range(100):
#     acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:False})
#     final_acu += acu
#   final_acu = final_acu / 100
#   print("final_acc: {}".format(final_acu))
#   predict = sess.run(prediction, feed_dict={x:my_data, keep_prob: 1, keep_prob_flatten: 1})
#   print("my_data shape: {}".format(my_data.shape))
#   print(predict[0])
#   print(type(acu))

training start
batch: 0, train_acc: 0.9700000286102295, avg_cost: 0.00014928001910448075
batch: 100, train_acc: 0.9599999785423279, avg_cost: 0.009403084375274678
batch: 200, train_acc: 0.9800000190734863, avg_cost: 0.01897650552292665
batch: 300, train_acc: 0.9399999976158142, avg_cost: 0.02932942595022421
batch: 400, train_acc: 0.9300000071525574, avg_cost: 0.03915649446037911
batch: 500, train_acc: 0.949999988079071, avg_cost: 0.04916275852359831
**************************************************
epoch: 0, train_acc: 0.9800000190734863, avg_cost: 0.059113255959625036
**************************************************
test_acc: 0.9300000059604645
batch: 0, train_acc: 0.9399999976158142, avg_cost: 9.919105097651482e-05
batch: 100, train_acc: 0.9700000286102295, avg_cost: 0.009492335151880972
batch: 200, train_acc: 0.9900000095367432, avg_cost: 0.018880791865910088
batch: 300, train_acc: 0.9599999785423279, avg_cost: 0.028198941253746543
batch: 400, train_acc: 0.9599999785423279, avg_c

# 32후 maxpool

In [0]:
'''
수정사항 
0. 18*2 / 18*2 / 18*2 resnet layer
1. 맨 처음 conv layer를 16 channel 단층으로 전환
2. #filter를 16-32-64로 전환
3. 마지막 maxpool 후 바로 average pool 적용
4. 마지막 fc에서 dropout 적용
'''

import numpy as np
import pandas as pd
import tensorflow as tf

tf.reset_default_graph()

learning_rate = 0.0001
training_epochs = 50
batch_size = 100

root = '/content/drive/My Drive/Colab Notebooks/MLpy'
load_file = 'model_mnist_fashion_cnn_ResNet_0815_BN_54l_bst_cost_3.ckpt'
load_file = root + load_file
save_file = root + 'model_mnist_fashion_cnn_ResNet_0815_BN_54l_bst_acc_3.ckpt'
save_file2 = root + 'model_mnist_fashion_cnn_ResNet_0815_BN_54l_bst_cost_3.ckpt'

x = tf.placeholder(tf.float32, [None,28,28])
x_img = tf.reshape(x, [-1, 28, 28, 1])
y = tf.placeholder(tf.int32, [None,1])
y_ohe = tf.reshape(tf.one_hot(y,10),[-1,10])

batch_prob = tf.placeholder(tf.bool)

keep_prob = tf.placeholder(tf.float32)
keep_prob_flatten = tf.placeholder(tf.float32)

h1 = make_conv_once_wo_pooling(x_img, 16, keep_prob, '1a', batch_prob)
h1 = tf.layers.batch_normalization(h1, center=True, scale=True, training=batch_prob)
h1 = make_conv_once_wo_pooling(h1, 16, keep_prob, '1b', batch_prob)
h1 = tf.layers.batch_normalization(h1, center=True, scale=True, training=batch_prob)
h1 = make_conv_once_wo_pooling(h1, 16, keep_prob, '1c', batch_prob)
h1 = tf.layers.batch_normalization(h1, center=True, scale=True, training=batch_prob)

iList1 = [16,16,16]
iList1_1 = [16,16,32]
iList2 = [32,32,32]
iList2_1 = [32,32,64]
iList3 = [64,64,64]
hidden_layer_list = []
h2 = my_resNet_model(h1, keep_prob, batch_prob, iList1, 2)
hidden_layer_list.append(h2)
for i in range(3,19):
  hl = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList1, i)
  hidden_layer_list.append(hl)

h19 = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList1_1, 19)
h19 = tf.nn.max_pool(h19, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
hidden_layer_list.append(h19)

for i in range(20,37):
  hl = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList2, i)
  hidden_layer_list.append(hl)

h37 = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList2_1, 37)
h37 = tf.nn.max_pool(h37, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
hidden_layer_list.append(h37)

for i in range(38,55):
  hl = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList3, i)
  hidden_layer_list.append(hl)

h55 = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList3, 55)
h55 = tf.nn.max_pool(h55, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
hidden_layer_list.append(h55)
  
h56 = tf.nn.avg_pool(h55, ksize=[1,7,7,1], strides=[1,7,7,1], padding="SAME")
h56_flat = tf.reshape(h56, [-1,64])

fcw1 = tf.get_variable("fcw1", shape=[64,10], initializer=tf.contrib.layers.xavier_initializer())
fcb1 = tf.zeros([10])

logits = tf.matmul(h56_flat,fcw1) + fcb1
logits = tf.nn.dropout(logits, keep_prob=keep_prob_flatten)

prediction = tf.argmax(logits, 1)

c_pre = tf.equal(tf.argmax(logits,1), tf.argmax(y_ohe,1))
acc = tf.reduce_mean(tf.cast(c_pre, tf.float32))

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_ohe, logits=logits))
train = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [0]:
print("training start")

accuracy_list = [0]
loss_list = [100]

import numpy as np

saver = tf.train.Saver()
my_data = np.zeros([1,28*28])

with tf.Session() as sess:
  saver.restore(sess, load_file)
#   sess.run(tf.global_variables_initializer())
  for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(train_images.shape[0]/batch_size)
    sf_images, sf_labels = shuffleData(train_images, train_labels)
    for i in range(total_batch):
      batch_xs, batch_ys = sf_images[batch_size*i:batch_size*(i+1),:,:], sf_labels[batch_size*i:batch_size*(i+1)]
      myfeed = {x:batch_xs, y:batch_ys, keep_prob:1, keep_prob_flatten:1, batch_prob:True}
      cv, accv,  _ = sess.run([cost, acc, train], feed_dict=myfeed)
      avg_cost += cv/total_batch
      if i%100 == 0:
        print("batch: {}, train_acc: {}, avg_cost: {}".format(i,accv,avg_cost))
    print("*"*50)  
    print("epoch: {}, train_acc: {}, avg_cost: {}".format(epoch,accv,avg_cost))
    print("*"*50)
    final_acu = 0
    for num in range(100):
      acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:True})
      final_acu += acu
    final_acu = final_acu / 100
    print("test_acc: {}".format(final_acu))
    if final_acu > accuracy_list[-1]:
      accuracy_list.append(final_acu)
      saver.save(sess, save_file)
    if avg_cost < loss_list[-1]:
      loss_list.append(avg_cost)
      saver.save(sess, save_file2)
      
print(accuracy_list)
print(loss_list)
  
#   final_acu = 0
#   for num in range(100):
#     acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:False})
#     final_acu += acu
#   final_acu = final_acu / 100
#   print("final_acc: {}".format(final_acu))
#   predict = sess.run(prediction, feed_dict={x:my_data, keep_prob: 1, keep_prob_flatten: 1})
#   print("my_data shape: {}".format(my_data.shape))
#   print(predict[0])
#   print(type(acu))

training start
batch: 0, train_acc: 0.9100000262260437, avg_cost: 0.00038107750316460927
batch: 100, train_acc: 0.949999988079071, avg_cost: 0.03238856510569651
batch: 200, train_acc: 0.8600000143051147, avg_cost: 0.0672397422045469
batch: 300, train_acc: 0.9300000071525574, avg_cost: 0.10062435284256925
batch: 400, train_acc: 0.9200000166893005, avg_cost: 0.13650368046015499
batch: 500, train_acc: 0.9599999785423279, avg_cost: 0.17484475293507182
**************************************************
epoch: 0, train_acc: 0.8799999952316284, avg_cost: 0.21053836705784007
**************************************************
test_acc: 0.8745999991893768
batch: 0, train_acc: 0.9300000071525574, avg_cost: 0.0004257612923781077
batch: 100, train_acc: 0.9300000071525574, avg_cost: 0.02984030584494274
batch: 200, train_acc: 0.9700000286102295, avg_cost: 0.060500100540618114
batch: 300, train_acc: 0.949999988079071, avg_cost: 0.09061081683884063
batch: 400, train_acc: 0.9599999785423279, avg_cost: 0

# Broad Architecture

In [0]:
'''
수정사항 
0. 18*2 / 18*2 / 18*2 resnet layer
1. 맨 처음 conv layer를 32 channel 단층으로 전환
2. #filter를 32-64-128로 전환
3. 마지막 maxpool 후 바로 average pool 적용
4. 마지막 fc에서 dropout 적용
'''

import numpy as np
import pandas as pd
import tensorflow as tf

tf.reset_default_graph()

learning_rate = 0.0001
training_epochs = 50
batch_size = 100

root = '/content/drive/My Drive/Colab Notebooks/MLpy'
load_file = 'model_mnist_fashion_cnn_ResNet_0815_BN_54l_bst_cost.ckpt'
load_file = root + load_file
save_file = root + 'model_mnist_fashion_cnn_ResNet_0815_BN_54l_bst_acc.ckpt'
save_file2 = root + 'model_mnist_fashion_cnn_ResNet_0815_BN_54l_bst_cost.ckpt'

x = tf.placeholder(tf.float32, [None,28,28])
x_img = tf.reshape(x, [-1, 28, 28, 1])
y = tf.placeholder(tf.int32, [None,1])
y_ohe = tf.reshape(tf.one_hot(y,10),[-1,10])

batch_prob = tf.placeholder(tf.bool)

keep_prob = tf.placeholder(tf.float32)
keep_prob_flatten = tf.placeholder(tf.float32)

h1 = make_conv_once_wo_pooling(x_img, 32, keep_prob, '1a', batch_prob)
h1 = tf.layers.batch_normalization(h1, center=True, scale=True, training=batch_prob)

iList1 = [32,32,32]
iList1_1 = [32,32,64]
iList2 = [64,64,64]
iList2_1 = [64,64,128]
iList3 = [128,128,128]
hidden_layer_list = []
h2 = my_resNet_model(h1, keep_prob, batch_prob, iList1, 2)
hidden_layer_list.append(h2)
for i in range(3,19):0
  hl = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList1, i)
  hidden_layer_list.append(hl)

h19 = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList1_1, 19)
h19 = tf.nn.max_pool(h19, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
hidden_layer_list.append(h19)

for i in range(20,37):
  hl = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList2, i)
  hidden_layer_list.append(hl)

h37 = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList2_1, 37)
h37 = tf.nn.max_pool(h37, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
hidden_layer_list.append(h37)

for i in range(38,55):
  hl = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList3, i)
  hidden_layer_list.append(hl)

h55 = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList3, 55)
h55 = tf.nn.max_pool(h55, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
hidden_layer_list.append(h55)
  
h56 = tf.nn.avg_pool(h55, ksize=[1,7,7,1], strides=[1,7,7,1], padding="SAME")
h56_flat = tf.reshape(h56, [-1,128])

fcw1 = tf.get_variable("fcw1", shape=[128,10], initializer=tf.contrib.layers.xavier_initializer())
fcb1 = tf.zeros([10])

logits = tf.matmul(h56_flat,fcw1) + fcb1
logits = tf.nn.dropout(logits, keep_prob=keep_prob_flatten)

prediction = tf.argmax(logits, 1)

c_pre = tf.equal(tf.argmax(logits,1), tf.argmax(y_ohe,1))
acc = tf.reduce_mean(tf.cast(c_pre, tf.float32))

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_ohe, logits=logits))
train = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [0]:
print("training start")

accuracy_list = [0]
loss_list = [100]

import numpy as np

saver = tf.train.Saver()
my_data = np.zeros([1,28*28])

with tf.Session() as sess:
#   saver.restore(sess, load_file)
  sess.run(tf.global_variables_initializer())
  for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(train_images.shape[0]/batch_size)
    sf_images, sf_labels = shuffleData(train_images, train_labels)
    for i in range(total_batch):
      batch_xs, batch_ys = sf_images[batch_size*i:batch_size*(i+1),:,:], sf_labels[batch_size*i:batch_size*(i+1)]
      myfeed = {x:batch_xs, y:batch_ys, keep_prob:1, keep_prob_flatten:0.9, batch_prob:True}
      cv, accv,  _ = sess.run([cost, acc, train], feed_dict=myfeed)
      avg_cost += cv/total_batch
      if i%100 == 0:
        print("batch: {}, train_acc: {}, avg_cost: {}".format(i,accv,avg_cost))
    print("*"*50)  
    print("epoch: {}, train_acc: {}, avg_cost: {}".format(epoch,accv,avg_cost))
    print("*"*50)
    final_acu = 0
    for num in range(100):
      acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:True})
      final_acu += acu
    final_acu = final_acu / 100
    print("test_acc: {}".format(final_acu))
    if final_acu > accuracy_list[-1]:
      accuracy_list.append(final_acu)
      saver.save(sess, save_file)
    if avg_cost < loss_list[-1]:
      loss_list.append(avg_cost)
      saver.save(sess, save_file2)
      
print(accuracy_list)
print(loss_list)
  
#   final_acu = 0
#   for num in range(100):
#     acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:False})
#     final_acu += acu
#   final_acu = final_acu / 100
#   print("final_acc: {}".format(final_acu))
#   predict = sess.run(prediction, feed_dict={x:my_data, keep_prob: 1, keep_prob_flatten: 1})
#   print("my_data shape: {}".format(my_data.shape))
#   print(predict[0])
#   print(type(acu))

training start
batch: 0, train_acc: 0.14000000059604645, avg_cost: 0.016808791160583494
batch: 100, train_acc: 0.7599999904632568, avg_cost: 0.25777745485305803
batch: 200, train_acc: 0.7599999904632568, avg_cost: 0.3869866437713308
batch: 300, train_acc: 0.6399999856948853, avg_cost: 0.5061947311460975
batch: 400, train_acc: 0.7599999904632568, avg_cost: 0.6131939578553041
batch: 500, train_acc: 0.7900000214576721, avg_cost: 0.7092880363762378
**************************************************
epoch: 0, train_acc: 0.8199999928474426, avg_cost: 0.804229662021002
**************************************************
test_acc: 0.8224999976158142
batch: 0, train_acc: 0.7599999904632568, avg_cost: 0.0012449613213539124
batch: 100, train_acc: 0.7599999904632568, avg_cost: 0.09131325448552764
batch: 200, train_acc: 0.8799999952316284, avg_cost: 0.17208044076959295
batch: 300, train_acc: 0.8600000143051147, avg_cost: 0.24883734159171586
batch: 400, train_acc: 0.7799999713897705, avg_cost: 0.3284

# 152

In [0]:
'''
수정사항 
0. 18*2 / 18*2 / 18*2 resnet layer
1. 맨 처음 conv layer를 16 channel 단층으로 전환
2. #filter를 16-32-64로 전환
3. 마지막 maxpool 후 바로 average pool 적용
4. 마지막 fc에서 dropout 적용
'''

import numpy as np
import pandas as pd
import tensorflow as tf

tf.reset_default_graph()

learning_rate = 0.00001
training_epochs = 50
batch_size = 100

root = '/content/drive/My Drive/Colab Notebooks/MLpy'
load_file = 'model_mnist_fashion_cnn_ResNet_0816_BN_152_bst_cost.ckpt'
load_file = root + load_file
save_file = root + 'model_mnist_fashion_cnn_ResNet_0816_BN_152_bst_acc.ckpt'
save_file2 = root + 'model_mnist_fashion_cnn_ResNet_0816_BN_152_bst_cost.ckpt'

x = tf.placeholder(tf.float32, [None,28,28])
x_img = tf.reshape(x, [-1, 28, 28, 1])
y = tf.placeholder(tf.int32, [None,1])
y_ohe = tf.reshape(tf.one_hot(y,10),[-1,10])

batch_prob = tf.placeholder(tf.bool)

keep_prob = tf.placeholder(tf.float32)
keep_prob_flatten = tf.placeholder(tf.float32)

h1 = make_conv_once_wo_pooling(x_img, 16, keep_prob, '1a', batch_prob)
h1 = tf.layers.batch_normalization(h1, center=True, scale=True, training=batch_prob)

iList1 = [16,16,16]
iList1_1 = [16,16,32]
iList2 = [32,32,32]
iList2_1 = [32,32,64]
iList3 = [64,64,64]
iList3_1 = [64,64,128]
iList4 = [128,128,128]
iList4_1 = [128,128,256]
iList5 = [256,256,256]
hidden_layer_list = []
h2 = my_resNet_model(h1, keep_prob, batch_prob, iList1, 2)
hidden_layer_list.append(h2)
for i in range(3,19):
  hl = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList1, i)
  hidden_layer_list.append(hl)

h19 = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList1_1, 19)
hidden_layer_list.append(h19)

for i in range(20,37):
  hl = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList2, i)
  hidden_layer_list.append(hl)

h37 = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList2_1, 37)
h37 = tf.nn.max_pool(h37, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
hidden_layer_list.append(h37)

for i in range(38,55):
  hl = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList3, i)
  hidden_layer_list.append(hl)

h55 = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList3_1, 55)
hidden_layer_list.append(h55)

for i in range(56,73):
  hl = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList4, i)
  hidden_layer_list.append(hl)
  
h73 = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList4_1, 73)
h73 = tf.nn.max_pool(h73, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
hidden_layer_list.append(h73)

for i in range(74,91):
  hl = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList5, i)
  hidden_layer_list.append(hl)
  
h91 = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList4_1, 91)
hidden_layer_list.append(h91)
  
h91 = tf.nn.avg_pool(h91, ksize=[1,7,7,1], strides=[1,7,7,1], padding="SAME")
h91_flat = tf.reshape(h91, [-1,256])

fcw1 = tf.get_variable("fcw1", shape=[256,10], initializer=tf.contrib.layers.xavier_initializer())
fcb1 = tf.zeros([10])

logits = tf.matmul(h91_flat,fcw1) + fcb1
logits = tf.nn.dropout(logits, keep_prob=keep_prob_flatten)

prediction = tf.argmax(logits, 1)

c_pre = tf.equal(tf.argmax(logits,1), tf.argmax(y_ohe,1))
acc = tf.reduce_mean(tf.cast(c_pre, tf.float32))

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_ohe, logits=logits))
train = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [10]:
print("training start")

accuracy_list = [0]
loss_list = [100]

import numpy as np

saver = tf.train.Saver()
my_data = np.zeros([1,28*28])

with tf.Session() as sess:
  saver.restore(sess, load_file)
#   sess.run(tf.global_variables_initializer())
  for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(train_images.shape[0]/batch_size)
    sf_images, sf_labels = shuffleData(train_images, train_labels)
    for i in range(total_batch):
      batch_xs, batch_ys = sf_images[batch_size*i:batch_size*(i+1),:,:], sf_labels[batch_size*i:batch_size*(i+1)]
      myfeed = {x:batch_xs, y:batch_ys, keep_prob:1, keep_prob_flatten:1, batch_prob:True}
      cv, accv,  _ = sess.run([cost, acc, train], feed_dict=myfeed)
      avg_cost += cv/total_batch
      if i%100 == 0:
        print("batch: {}, train_acc: {}, avg_cost: {}".format(i,accv,avg_cost))
    print("*"*50)  
    print("epoch: {}, train_acc: {}, avg_cost: {}".format(epoch,accv,avg_cost))
    print("*"*50)
    final_acu = 0
    for num in range(100):
      acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:True})
      final_acu += acu
    final_acu = final_acu / 100
    print("test_acc: {}".format(final_acu))
    if final_acu > accuracy_list[-1]:
      accuracy_list.append(final_acu)
      saver.save(sess, save_file)
    if avg_cost < loss_list[-1]:
      loss_list.append(avg_cost)
      saver.save(sess, save_file2)
      
print(accuracy_list)
print(loss_list)
  
#   final_acu = 0
#   for num in range(100):
#     acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:False})
#     final_acu += acu
#   final_acu = final_acu / 100
#   print("final_acc: {}".format(final_acu))
#   predict = sess.run(prediction, feed_dict={x:my_data, keep_prob: 1, keep_prob_flatten: 1})
#   print("my_data shape: {}".format(my_data.shape))
#   print(predict[0])
#   print(type(acu))

training start
batch: 0, train_acc: 1.0, avg_cost: 1.012995218237241e-05
batch: 100, train_acc: 0.9900000095367432, avg_cost: 0.0036641801839383944
batch: 200, train_acc: 0.9900000095367432, avg_cost: 0.006155472416721749
batch: 300, train_acc: 0.9900000095367432, avg_cost: 0.008049827112893884
batch: 400, train_acc: 0.9900000095367432, avg_cost: 0.009994200265694713
batch: 500, train_acc: 1.0, avg_cost: 0.011806211970251746
**************************************************
epoch: 0, train_acc: 1.0, avg_cost: 0.013510935285400283
**************************************************
test_acc: 0.9114000028371811
batch: 0, train_acc: 1.0, avg_cost: 3.074880223721266e-06
batch: 100, train_acc: 1.0, avg_cost: 0.000763924408723445
batch: 200, train_acc: 1.0, avg_cost: 0.001536541542567041
batch: 300, train_acc: 1.0, avg_cost: 0.0022444882391088566
batch: 400, train_acc: 1.0, avg_cost: 0.00279537334189323
batch: 500, train_acc: 1.0, avg_cost: 0.0035273198499635294
*****************************

KeyboardInterrupt: ignored

# WRN : for 94% (too much big scale)

In [0]:
'''
수정사항 
0. 28*2 / 28*2 / 28*2 wide resnet layer
1. 맨 처음 conv layer를 160 channel 단층으로 전환
2. #filter를 160-320-640로 전환
3. 마지막 maxpool 후 바로 average pool 적용
4. 마지막 fc에서 dropout 적용
5. 1epoch당 60000/50 = 1200iteration
'''

import numpy as np
import pandas as pd
import tensorflow as tf

tf.reset_default_graph()

learning_rate = 0.001
training_epochs = 50
batch_size = 50

root = '/content/drive/My Drive/Colab Notebooks/MLpy'
load_file = 'model_mnist_fashion_cnn_WRN_0816_BN_cost.ckpt'
load_file = root + load_file
save_file = root + 'model_mnist_fashion_cnn_WRN_0816_BN_acc.ckpt'
save_file2 = root + 'model_mnist_fashion_cnn_WRN_0816_BN_cost.ckpt'

x = tf.placeholder(tf.float32, [None,28,28])
x_img = tf.reshape(x, [-1, 28, 28, 1])
y = tf.placeholder(tf.int32, [None,1])
y_ohe = tf.reshape(tf.one_hot(y,10),[-1,10])

batch_prob = tf.placeholder(tf.bool)

keep_prob = tf.placeholder(tf.float32)
keep_prob_flatten = tf.placeholder(tf.float32)

h1 = make_conv_once_wo_pooling(x_img, 160, keep_prob, '1a', batch_prob)
h1 = tf.layers.batch_normalization(h1, center=True, scale=True, training=batch_prob)

iList1 = [160,160,160]
iList1_1 = [160,160,320]
iList2 = [320,320,320]
iList2_1 = [320,320,640]
iList3 = [640,640,640]
hidden_layer_list = []
h2 = my_resNet_model(h1, keep_prob, batch_prob, iList1, 2)
hidden_layer_list.append(h2)
for i in range(3,29):
  hl = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList1, i)
  hidden_layer_list.append(hl)

h29 = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList1_1, 29)
h29 = tf.nn.max_pool(h29, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
hidden_layer_list.append(h29)

for i in range(30,57):
  hl = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList2, i)
  hidden_layer_list.append(hl)

h57 = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList2_1, 57)
h57 = tf.nn.max_pool(h57, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
hidden_layer_list.append(h57)

for i in range(58,85):
  hl = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList3, i)
  hidden_layer_list.append(hl)

h85 = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList3, 85)
h85 = tf.nn.max_pool(h85, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
hidden_layer_list.append(h85)
  
h86 = tf.nn.avg_pool(h85, ksize=[1,7,7,1], strides=[1,7,7,1], padding="SAME")
h86_flat = tf.reshape(h86, [-1,640])

fcw1 = tf.get_variable("fcw1", shape=[640,10], initializer=tf.contrib.layers.xavier_initializer())
fcb1 = tf.zeros([10])

logits = tf.matmul(h86_flat,fcw1) + fcb1
logits = tf.nn.dropout(logits, keep_prob=keep_prob_flatten)

prediction = tf.argmax(logits, 1)

c_pre = tf.equal(tf.argmax(logits,1), tf.argmax(y_ohe,1))
acc = tf.reduce_mean(tf.cast(c_pre, tf.float32))

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_ohe, logits=logits))
train = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [22]:
print("training start")

accuracy_list = [0]
loss_list = [100]

import numpy as np

saver = tf.train.Saver()
my_data = np.zeros([1,28*28])

with tf.Session() as sess:
  saver.restore(sess, load_file)
#   sess.run(tf.global_variables_initializer())
  for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(train_images.shape[0]/batch_size)
    sf_images, sf_labels = shuffleData(train_images, train_labels)
    for i in range(total_batch):
      batch_xs, batch_ys = sf_images[batch_size*i:batch_size*(i+1),:,:], sf_labels[batch_size*i:batch_size*(i+1)]
      myfeed = {x:batch_xs, y:batch_ys, keep_prob:0.7, keep_prob_flatten:1, batch_prob:True}
      cv, accv,  _ = sess.run([cost, acc, train], feed_dict=myfeed)
      avg_cost += cv/total_batch
      if i%100 == 0:
        print("batch: {}, train_acc: {}, avg_cost: {}".format(i,accv,avg_cost))
    print("*"*50)
    print("epoch: {}, train_acc: {}, avg_cost: {}".format(epoch,accv,avg_cost))
    print("*"*50)
    final_acu = 0
    for num in range(200):
      acu = sess.run(acc, feed_dict={x:test_images[50*num:50*(num+1),:,:],y:test_labels[50*num:50*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:True})
      final_acu += acu
    final_acu = final_acu / 200
    print("test_acc: {}".format(final_acu))
    if final_acu > accuracy_list[-1]:
      accuracy_list.append(final_acu)
      saver.save(sess, save_file)
    if avg_cost < loss_list[-1]:
      loss_list.append(avg_cost)
      saver.save(sess, save_file2)
      
print(accuracy_list)
print(loss_list)
  
#   final_acu = 0
#   for num in range(100):
#     acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:False})
#     final_acu += acu
#   final_acu = final_acu / 100
#   print("final_acc: {}".format(final_acu))
#   predict = sess.run(prediction, feed_dict={x:my_data, keep_prob: 1, keep_prob_flatten: 1})
#   print("my_data shape: {}".format(my_data.shape))
#   print(predict[0])
#   print(type(acu))

training start
batch: 0, train_acc: 0.7400000095367432, avg_cost: 0.0021068376302719118
batch: 100, train_acc: 0.7400000095367432, avg_cost: 0.07177234232425686
batch: 200, train_acc: 0.7799999713897705, avg_cost: 0.13477707090477156
batch: 300, train_acc: 0.7799999713897705, avg_cost: 0.1968955680976312
batch: 400, train_acc: 0.800000011920929, avg_cost: 0.2590624453375737
batch: 500, train_acc: 0.800000011920929, avg_cost: 0.3190614515294632
batch: 600, train_acc: 0.7400000095367432, avg_cost: 0.37459514717261005
batch: 700, train_acc: 0.6800000071525574, avg_cost: 0.43008347928524004
batch: 800, train_acc: 0.7599999904632568, avg_cost: 0.4855405419816572
batch: 900, train_acc: 0.7200000286102295, avg_cost: 0.5427726601312554
batch: 1000, train_acc: 0.7799999713897705, avg_cost: 0.5996599838634328
batch: 1100, train_acc: 0.7599999904632568, avg_cost: 0.6555540790408846
**************************************************
epoch: 0, train_acc: 0.7799999713897705, avg_cost: 0.712572303016

KeyboardInterrupt: ignored

# Modified WRN

In [0]:
'''
수정사항 
0. 28*2 / 28*2 / 28*2 wide resnet layer
1. 맨 처음 conv layer를 160 channel 단층으로 전환
2. #filter를 160-320-640로 전환
3. 마지막 maxpool 후 바로 average pool 적용
4. 마지막 fc에서 dropout 적용
5. 1epoch당 60000/50 = 1200iteration
'''

import numpy as np
import pandas as pd
import tensorflow as tf

tf.reset_default_graph()

learning_rate = 0.001
training_epochs = 50
batch_size = 50

root = '/content/drive/My Drive/Colab Notebooks/MLpy'
load_file = 'model_mnist_fashion_cnn_WRN_revised_0816_BN_cost.ckpt'
load_file = root + load_file
save_file = root + 'model_mnist_fashion_cnn_WRN_revised_0816_BN_acc.ckpt'
save_file2 = root + 'model_mnist_fashion_cnn_WRN_revised_0816_BN_cost.ckpt'

x = tf.placeholder(tf.float32, [None,28,28])
x_img = tf.reshape(x, [-1, 28, 28, 1])
y = tf.placeholder(tf.int32, [None,1])
y_ohe = tf.reshape(tf.one_hot(y,10),[-1,10])

batch_prob = tf.placeholder(tf.bool)

keep_prob = tf.placeholder(tf.float32)
keep_prob_flatten = tf.placeholder(tf.float32)

h1 = make_conv_once_wo_pooling(x_img, 160, keep_prob, '1a', batch_prob)
h1 = tf.layers.batch_normalization(h1, center=True, scale=True, training=batch_prob)

iList1 = [160,160,160]
iList1_1 = [160,160,320]
iList2 = [320,320,320]
iList2_1 = [320,320,640]
iList3 = [640,640,640]
hidden_layer_list = []
h2 = my_resNet_model(h1, keep_prob, batch_prob, iList1, 2)
hidden_layer_list.append(h2)
for i in range(3,5):
  hl = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList1, i)
  hidden_layer_list.append(hl)

h5 = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList1_1, 5)
h5 = tf.nn.max_pool(h5, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
hidden_layer_list.append(h5)

for i in range(6,9):
  hl = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList2, i)
  hidden_layer_list.append(hl)

h9 = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList2_1, 9)
h9 = tf.nn.max_pool(h9, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
hidden_layer_list.append(h9)

for i in range(10,13):
  hl = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList3, i)
  hidden_layer_list.append(hl)

h13 = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList3, 13)
h13 = tf.nn.max_pool(h13, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
hidden_layer_list.append(h13)
  
h14 = tf.nn.avg_pool(h13, ksize=[1,7,7,1], strides=[1,7,7,1], padding="SAME")
h14_flat = tf.reshape(h14, [-1,640])

fcw1 = tf.get_variable("fcw1", shape=[640,10], initializer=tf.contrib.layers.xavier_initializer())
fcb1 = tf.zeros([10])

logits = tf.matmul(h14_flat,fcw1) + fcb1
logits = tf.nn.dropout(logits, keep_prob=keep_prob_flatten)

prediction = tf.argmax(logits, 1)

c_pre = tf.equal(tf.argmax(logits,1), tf.argmax(y_ohe,1))
acc = tf.reduce_mean(tf.cast(c_pre, tf.float32))

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_ohe, logits=logits))
train = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [0]:
print("training start")

accuracy_list = [0]
loss_list = [100]

import numpy as np

saver = tf.train.Saver()
my_data = np.zeros([1,28*28])

with tf.Session() as sess:
#   saver.restore(sess, load_file)
  sess.run(tf.global_variables_initializer())
  for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(train_images.shape[0]/batch_size)
    sf_images, sf_labels = shuffleData(train_images, train_labels)
    for i in range(total_batch):
      batch_xs, batch_ys = sf_images[batch_size*i:batch_size*(i+1),:,:], sf_labels[batch_size*i:batch_size*(i+1)]
      myfeed = {x:batch_xs, y:batch_ys, keep_prob:0.7, keep_prob_flatten:1, batch_prob:True}
      cv, accv,  _ = sess.run([cost, acc, train], feed_dict=myfeed)
      avg_cost += cv/total_batch
      if i%100 == 0:
        print("batch: {}, train_acc: {}, avg_cost: {}".format(i,accv,avg_cost))
    print("*"*50)
    print("epoch: {}, train_acc: {}, avg_cost: {}".format(epoch,accv,avg_cost))
    print("*"*50)
    final_acu = 0
    for num in range(200):
      acu = sess.run(acc, feed_dict={x:test_images[50*num:50*(num+1),:,:],y:test_labels[50*num:50*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:True})
      final_acu += acu
    final_acu = final_acu / 200
    print("test_acc: {}".format(final_acu))
    if final_acu > accuracy_list[-1]:
      accuracy_list.append(final_acu)
      saver.save(sess, save_file)
    if avg_cost < loss_list[-1]:
      loss_list.append(avg_cost)
      saver.save(sess, save_file2)
      
print(accuracy_list)
print(loss_list)
  
#   final_acu = 0
#   for num in range(100):
#     acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:False})
#     final_acu += acu
#   final_acu = final_acu / 100
#   print("final_acc: {}".format(final_acu))
#   predict = sess.run(prediction, feed_dict={x:my_data, keep_prob: 1, keep_prob_flatten: 1})
#   print("my_data shape: {}".format(my_data.shape))
#   print(predict[0])
#   print(type(acu))

training start
batch: 0, train_acc: 0.11999999731779099, avg_cost: 0.0032632982730865477
batch: 100, train_acc: 0.7200000286102295, avg_cost: 1.1852482240895434
batch: 200, train_acc: 0.7599999904632568, avg_cost: 1.2946564383804804
batch: 300, train_acc: 0.6399999856948853, avg_cost: 1.495631261120241
batch: 400, train_acc: 0.7799999713897705, avg_cost: 1.5919538464893908
batch: 500, train_acc: 0.699999988079071, avg_cost: 1.6672483658045536
batch: 600, train_acc: 0.7799999713897705, avg_cost: 1.737454542542498
batch: 700, train_acc: 0.7400000095367432, avg_cost: 1.8170557390029227
batch: 800, train_acc: 0.7799999713897705, avg_cost: 1.8789849527304368
batch: 900, train_acc: 0.800000011920929, avg_cost: 1.9407696837559345
batch: 1000, train_acc: 0.7400000095367432, avg_cost: 1.9960565011699996
batch: 1100, train_acc: 0.8199999928474426, avg_cost: 2.0419485336293786
**************************************************
epoch: 0, train_acc: 0.8600000143051147, avg_cost: 2.089116011286775
*

# Restore : rl = 0.0001

In [0]:
'''
수정사항 
0. 28*2 / 28*2 / 28*2 wide resnet layer
1. 맨 처음 conv layer를 160 channel 단층으로 전환
2. #filter를 160-320-640로 전환
3. 마지막 maxpool 후 바로 average pool 적용
4. 마지막 fc에서 dropout 적용
5. 1epoch당 60000/50 = 1200iteration
'''

import numpy as np
import pandas as pd
import tensorflow as tf

tf.reset_default_graph()

learning_rate = 0.0001
training_epochs = 50
batch_size = 50

root = '/content/drive/My Drive/Colab Notebooks/MLpy'
load_file = 'model_mnist_fashion_cnn_WRN_revised_0816_BN_cost.ckpt'
load_file = root + load_file
save_file = root + 'model_mnist_fashion_cnn_WRN_revised_0816_BN_acc.ckpt'
save_file2 = root + 'model_mnist_fashion_cnn_WRN_revised_0816_BN_cost.ckpt'

x = tf.placeholder(tf.float32, [None,28,28])
x_img = tf.reshape(x, [-1, 28, 28, 1])
y = tf.placeholder(tf.int32, [None,1])
y_ohe = tf.reshape(tf.one_hot(y,10),[-1,10])

batch_prob = tf.placeholder(tf.bool)

keep_prob = tf.placeholder(tf.float32)
keep_prob_flatten = tf.placeholder(tf.float32)

h1 = make_conv_once_wo_pooling(x_img, 160, keep_prob, '1a', batch_prob)
h1 = tf.layers.batch_normalization(h1, center=True, scale=True, training=batch_prob)

iList1 = [160,160,160]
iList1_1 = [160,160,320]
iList2 = [320,320,320]
iList2_1 = [320,320,640]
iList3 = [640,640,640]
hidden_layer_list = []
h2 = my_resNet_model(h1, keep_prob, batch_prob, iList1, 2)
hidden_layer_list.append(h2)
for i in range(3,5):
  hl = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList1, i)
  hidden_layer_list.append(hl)

h5 = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList1_1, 5)
h5 = tf.nn.max_pool(h5, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
hidden_layer_list.append(h5)

for i in range(6,9):
  hl = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList2, i)
  hidden_layer_list.append(hl)

h9 = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList2_1, 9)
h9 = tf.nn.max_pool(h9, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
hidden_layer_list.append(h9)

for i in range(10,13):
  hl = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList3, i)
  hidden_layer_list.append(hl)

h13 = my_resNet_model(hidden_layer_list[-1], keep_prob, batch_prob, iList3, 13)
h13 = tf.nn.max_pool(h13, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
hidden_layer_list.append(h13)
  
h14 = tf.nn.avg_pool(h13, ksize=[1,7,7,1], strides=[1,7,7,1], padding="SAME")
h14_flat = tf.reshape(h14, [-1,640])

fcw1 = tf.get_variable("fcw1", shape=[640,10], initializer=tf.contrib.layers.xavier_initializer())
fcb1 = tf.zeros([10])

logits = tf.matmul(h14_flat,fcw1) + fcb1
logits = tf.nn.dropout(logits, keep_prob=keep_prob_flatten)

prediction = tf.argmax(logits, 1)

c_pre = tf.equal(tf.argmax(logits,1), tf.argmax(y_ohe,1))
acc = tf.reduce_mean(tf.cast(c_pre, tf.float32))

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_ohe, logits=logits))
train = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [0]:
print("training start")

accuracy_list = [0]
loss_list = [100]

import numpy as np

saver = tf.train.Saver()
my_data = np.zeros([1,28*28])

with tf.Session() as sess:
  saver.restore(sess, load_file)
#   sess.run(tf.global_variables_initializer())
  for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(train_images.shape[0]/batch_size)
    sf_images, sf_labels = shuffleData(train_images, train_labels)
    for i in range(total_batch):
      batch_xs, batch_ys = sf_images[batch_size*i:batch_size*(i+1),:,:], sf_labels[batch_size*i:batch_size*(i+1)]
      myfeed = {x:batch_xs, y:batch_ys, keep_prob:0.7, keep_prob_flatten:1, batch_prob:True}
      cv, accv,  _ = sess.run([cost, acc, train], feed_dict=myfeed)
      avg_cost += cv/total_batch
      if i%100 == 0:
        print("batch: {}, train_acc: {}, avg_cost: {}".format(i,accv,avg_cost))
    print("*"*50)
    print("epoch: {}, train_acc: {}, avg_cost: {}".format(epoch,accv,avg_cost))
    print("*"*50)
    final_acu = 0
    for num in range(200):
      acu = sess.run(acc, feed_dict={x:test_images[50*num:50*(num+1),:,:],y:test_labels[50*num:50*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:True})
      final_acu += acu
    final_acu = final_acu / 200
    print("test_acc: {}".format(final_acu))
    if final_acu > accuracy_list[-1]:
      accuracy_list.append(final_acu)
      saver.save(sess, save_file)
    if avg_cost < loss_list[-1]:
      loss_list.append(avg_cost)
      saver.save(sess, save_file2)
      
print(accuracy_list)
print(loss_list)
  
#   final_acu = 0
#   for num in range(100):
#     acu = sess.run(acc, feed_dict={x:test_images[100*num:100*(num+1),:,:],y:test_labels[100*num:100*(num+1),:],keep_prob:1,keep_prob_flatten:1,batch_prob:False})
#     final_acu += acu
#   final_acu = final_acu / 100
#   print("final_acc: {}".format(final_acu))
#   predict = sess.run(prediction, feed_dict={x:my_data, keep_prob: 1, keep_prob_flatten: 1})
#   print("my_data shape: {}".format(my_data.shape))
#   print(predict[0])
#   print(type(acu))